In [1]:
# Imports 
import numpy as np
from numpy import *
from scipy.special import erf
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from scipy.special import factorial2 as fact2
import itertools
import time
from numba import njit
#import numba_scipy
#import numba_special  # The import generates Numba overloads for special

In [2]:
# Molecule definition

molecule = 'H2O'

# Atomic valencies
Z = [1, 1, 8]

# Atom labels
atom_labels = ['H', 'H', 'O']

# Number of atoms
Nat = len(Z)

# Number of electrons
Nelec = int(sum(Z))     # neutral atom

basis = '6-31g'

In [3]:
### Calculation setup

# SCF solver parameters
level_shift = 0.0
damp_p = 2/3
maxit_scf = 500
tol_scf = 10**-9
diis_size_scf = 3
flag_scf_verbose = False

# CC Solver parameters
maxit_cc = 100
tol_cc = 10**-9
diis_size_cc = 5
flag_cc_verbose = True

# EOM-CC Solver parameters
# Tune number of vectors per state and thresh_vec to prevent eigenvalue collapse to 0
# Most important thing is getting good initial guess. Needs CISD in small active space. Pick states based on 
# orbital or molecular symmetry... 
maxit_eom = 80
tol_eom = 10**-4
nstates = 5
nvec_per_state = 2
max_nvec_per_state = 30
thresh_vec = 10**-8
flag_eom_verbose = True
flag_cis_guess = False 
eigensolver = 1
root_homing = 'energy'

# Molecular geometries
num_pts = 1
atom_coordinates = []
#RJ = np.linspace(1.3983972315484032,1.3983972315484032,num_pts)
RJ = np.linspace(0.5,5,num_pts)
HOH_angle = np.radians([104.5])

for J in range(num_pts):

    if molecule== 'H2O' or molecule=='SH2':
        theta = 0.5*(pi - HOH_angle)
        atom_coordinates.append( [[RJ[J]*np.cos(theta), RJ[J]*np.sin(theta), 0],
                                 [-RJ[J]*np.cos(theta), RJ[J]*np.sin(theta), 0],
                                 [0, 0, 0]] )
    else:
        atom_coordinates.append( [[0, 0, -RJ[J]/2],
                                  [0, 0, RJ[J]/2]] )


In [4]:

# spec = [
#         ("origin", float32[:]),
#         ("shell", float32[:]),
#         ("exps", tuple)
#         ("coefs", float32[:])
#         ("norm", float32)
#        ]

#@jit(nopython=True)
class BasisFunction:
    ''' A class that contains all our basis function data
            Attributes:
            origin: array/list containing the coordinates of the Gaussian origin
            shell:  tuple of angular momentum
            exps:   list of primitive Gaussian exponents
            coefs:  list of primitive Gaussian coefficients
            norm:   list of normalization factors for Gaussian primitives
    '''
    
    
    def __init__(self,origin=[0.0,0.0,0.0],shell=(0,0,0),exps=[],coefs=[]): 
        self.origin = np.asarray(origin)
        self.shell = shell
        self.exps = exps
        self.coefs = coefs
        self.norm = None
        self.normalize()
        
    
    def normalize(self):
        ''' Routine to normalize the basis functions, in case they
                    do not integrate to unity.
                '''
        l,m,n = self.shell
        L = l+m+n
        # self.norm is a list of length equal to number primitives 
        # normalize primitives first (PGBFs)
        self.norm = np.sqrt(np.power(2,2*(l+m+n)+1.5)*\
                            np.power(self.exps,l+m+n+1.5)/fact2(2*l-1)/fact2(2*m-1)/fact2(2*n-1)/np.power(np.pi,1.5))
        # now normalize the contracted basis functions (CGBFs) # Eq. 1.44 of Valeev integral whitepaper
        prefactor = np.power(np.pi,1.5)*\
                fact2(2*l - 1)*fact2(2*m - 1)*fact2(2*n - 1)/np.power(2.0,L)
        N = 0.0
        num_exps = len(self.exps) 
        for ia in range(num_exps):
            for ib in range(num_exps):
                N += self.norm[ia]*self.norm[ib]*self.coefs[ia]*self.coefs[ib]/np.power(self.exps[ia] + self.exps[ib],L+1.5)
        N *= prefactor
        N = np.power(N,-0.5)
        for ia in range(num_exps):
            self.coefs[ia] *= N
        
   
    def basisfcn_matrix(self,xdom,ydom,zdom):
        ''' Method that returns a 3-dimensional matrix corresponding to the 
            3D basis function evaluated in space on over a domain (xdom,ydom,zdom)
        '''
        l,m,n = self.shell
        xi = self.exps  
        G = np.zeros((len(xdom),len(ydom),len(zdom)))
        for i in range(len(xi)):
            gauss_x = xdom**l*np.exp(-xi[i]*(xdom-self.origin[0])**2)
            gauss_y = ydom**m*np.exp(-xi[i]*(ydom-self.origin[1])**2)
            gauss_z = zdom**n*np.exp(-xi[i]*(zdom-self.origin[2])**2)
            Temp = np.kron(gauss_z,np.kron(gauss_y,gauss_x))
            G += self.coefs[i]*np.reshape(Temp,[len(xdom),len(ydom),len(zdom)])
            
        return G
    
            
            
            

In [5]:
#@njit
def E(i,j,t,Qx,a,b):
    ''' Recursive definition of Hermite Gaussian coefficients.
            Returns a float.
            a: orbital exponent on Gaussian 'a' (e.g. alpha in the text)
            b: orbital exponent on Gaussian 'b' (e.g. beta in the text)
            i,j: orbital angular momentum number on Gaussian 'a' and 'b'
            t: number nodes in Hermite (depends on type of integral,
               e.g. always zero for overlap integrals)
            Qx: distance between origins of Gaussian 'a' and 'b'
    '''
    p=a+b
    q = a*b/p
    if (t < 0) or (t > (i + j)): # out of bounds for t
        return 0.0
    elif i == j == t == 0: # base case
        return np.exp(-q*Qx*Qx) # K_AB 
    elif j == 0: # decrement index i
        return (1/(2*p))*E(i-1,j,t-1,Qx,a,b) -  (q*Qx/a)*E(i-1,j,t,Qx,a,b) +  (t+1)*E(i-1,j,t+1,Qx,a,b)
    else: # decrement index j
        return (1/(2*p))*E(i,j-1,t-1,Qx,a,b) +  (q*Qx/b)*E(i,j-1,t,Qx,a,b) +  (t+1)*E(i,j-1,t+1,Qx,a,b)

def overlap(a,lmn1,A,b,lmn2,B):
    ''' Evaluates overlap integral between two Gaussians
            Returns a float.
            a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
            b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
            lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
                  for Gaussian 'a'
            lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
            A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
            B:    list containing origin of Gaussian 'b'
    '''
    l1,m1,n1 = lmn1 # shell angular momentum on Gaussian 'a' 
    l2,m2,n2 = lmn2 # shell angular momentum on Gaussian 'b' 
    S1 = E(l1,l2,0,A[0]-B[0],a,b) # X
    S2 = E(m1,m2,0,A[1]-B[1],a,b) # Y
    S3 = E(n1,n2,0,A[2]-B[2],a,b) # Z
    return S1*S2*S3*np.power(np.pi/(a+b),1.5)


def kinetic(a,lmn1,A,b,lmn2,B):
    ''' Evaluates kinetic energy integral between two Gaussians
            Returns a float.
            a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
            b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
            lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
                  for Gaussian 'a'
            lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
            A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
            B:    list containing origin of Gaussian 'b'
    '''
    
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    
    term0 = b*(2*(l2+m2+n2)+3)*overlap(a,(l1,m1,n1),A,b,(l2,m2,n2),B)
    
    term1 = -2*np.power(b,2)*(overlap(a,(l1,m1,n1),A,b,(l2+2,m2,n2),B) +
                            overlap(a,(l1,m1,n1),A,b,(l2,m2+2,n2),B) +
                            overlap(a,(l1,m1,n1),A,b,(l2,m2,n2+2),B))
    
    term2 = -0.5*(l2*(l2-1)*overlap(a,(l1,m1,n1),A,b,(l2-2,m2,n2),B) +
                  m2*(m2-1)*overlap(a,(l1,m1,n1),A,b,(l2,m2-2,n2),B) +
                  n2*(n2-1)*overlap(a,(l1,m1,n1),A,b,(l2,m2,n2-2),B)) 
    
    return term0+term1+term2

#@jit(nopython=True)
def R(t,u,v,n,p,PCx,PCy,PCz,RPC):
    ''' Returns the Coulomb auxiliary Hermite integrals
            Returns a float.
            Arguments:
            t,u,v:   order of Coulomb Hermite derivative in x,y,z
                     (see defs in Helgaker and Taylor)
            n:       order of Boys function
            PCx,y,z: Cartesian vector distance between Gaussian
                     composite center P and nuclear center C
            RPC:     Distance between P and C
    '''
    T = p*RPC*RPC
    val = 0.0
    if t == u == v == 0:
        val += np.power(-2*p,n)*boys(float(n),T) 
    elif t == u == 0:
        if v > 1:
            val += (v-1)*R(t,u,v-2,n+1,p,PCx,PCy,PCz,RPC)
        val += PCz*R(t,u,v-1,n+1,p,PCx,PCy,PCz,RPC) 
    elif t == 0:
        if u > 1:
            val += (u-1)*R(t,u-2,v,n+1,p,PCx,PCy,PCz,RPC)
        val += PCy*R(t,u-1,v,n+1,p,PCx,PCy,PCz,RPC) 
    else:
        if t > 1:
            val += (t-1)*R(t-2,u,v,n+1,p,PCx,PCy,PCz,RPC)
        val += PCx*R(t-1,u,v,n+1,p,PCx,PCy,PCz,RPC) 
    return val

from scipy.special import hyp1f1
#@jit(nopython=True)
def boys(n,T):
    return hyp1f1(n+0.5,n+1.5,-T)/(2.0*n+1.0)

#@jit(nopython=True)
def gaussian_product_center(a,A,b,B): 
    return (a*A+b*B)/(a+b)


def nuclear_attraction(a,lmn1,A,b,lmn2,B,C):
    ''' Evaluates kinetic energy integral between two Gaussians
             Returns a float.
             a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
             b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
             lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
                   for Gaussian 'a'
             lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
             A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
             B:    list containing origin of Gaussian 'b'
             C:    list containing origin of nuclear center 'C'
    '''
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    p=a+b
    P = gaussian_product_center(a,A,b,B) # Gaussian composite center 
    RPC = np.linalg.norm(P-C)
    val = 0.0
    for t in range(l1+l2+1):
        for u in range(m1+m2+1):
            for v in range(n1+n2+1):
                 val += E(l1,l2,t,A[0]-B[0],a,b) * E(m1,m2,u,A[1]-B[1],a,b) * E(n1,n2,v,A[2]-B[2],a,b) * \
                    R(t,u,v,0,p,P[0]-C[0],P[1]-C[1],P[2]-C[2],RPC)
    val *= 2*np.pi/p 
    return val

#@jit(nopython=True)
def electron_repulsion(a,lmn1,A,b,lmn2,B,c,lmn3,C,d,lmn4,D): 
    ''' Evaluates kinetic energy integral between two Gaussians
         Returns a float.
         a,b,c,d:   orbital exponent on Gaussian 'a','b','c','d'
         lmn1,lmn2
         lmn3,lmn4: int tuple containing orbital angular momentum
                    for Gaussian 'a','b','c','d', respectively
         A,B,C,D:   list containing origin of Gaussian 'a','b','c','d'
    '''
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    l3,m3,n3 = lmn3
    l4,m4,n4 = lmn4
    p = a+b # composite exponent for P (from Gaussians 'a' and 'b') 
    q = c+d # composite exponent for Q (from Gaussians 'c' and 'd') 
    alpha = p*q/(p+q)
    P = gaussian_product_center(a,A,b,B) # A and B composite center 
    Q = gaussian_product_center(c,C,d,D) # C and D composite center 
    RPQ = np.linalg.norm(P-Q)
    val = 0.0
    for t in range(l1+l2+1):
        for u in range(m1+m2+1):
            for v in range(n1+n2+1):
                for tau in range(l3+l4+1): 
                    for nu in range(m3+m4+1):
                        for phi in range(n3+n4+1):
                            val += E(l1,l2,t,A[0]-B[0],a,b) * \
                                    E(m1,m2,u,A[1]-B[1],a,b) * \
                                    E(n1,n2,v,A[2]-B[2],a,b) * \
                                    E(l3,l4,tau,C[0]-D[0],c,d) * \
                                    E(m3,m4,nu ,C[1]-D[1],c,d) * \
                                    E(n3,n4,phi,C[2]-D[2],c,d) * \
                                    np.power(-1,tau+nu+phi) * \
                                    R(t+tau,u+nu,v+phi,0,\
                                        alpha,P[0]-Q[0],P[1]-Q[1],P[2]-Q[2],RPQ)
    val *= 2*np.power(np.pi,2.5)/(p*q*np.sqrt(p+q)) 
    return val

In [6]:

def S(a,b):
    '''Evaluates overlap between two contracted Gaussians
           Returns float.
           Arguments:
           a: contracted Gaussian 'a', BasisFunction object
           b: contracted Gaussian 'b', BasisFunction object
    '''
    s = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            s += a.norm[ia]*b.norm[ib]*ca*cb*\
                 overlap(a.exps[ia],a.shell,a.origin,
                 b.exps[ib],b.shell,b.origin)
    return s


def T(a,b):
    '''Evaluates kinetic energy between two contracted Gaussians
           Returns float.
           Arguments:
           a: contracted Gaussian 'a', BasisFunction object
           b: contracted Gaussian 'b', BasisFunction object
    '''
    t = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            t += a.norm[ia]*b.norm[ib]*ca*cb*kinetic(a.exps[ia],a.shell,a.origin,b.exps[ib],b.shell,b.origin)
    return t


def V(a,b,C,Z):
    '''Evaluates overlap between two contracted Gaussians
            Returns float.
            Arguments:
            a: contracted Gaussian 'a', BasisFunction object
            b: contracted Gaussian 'b', BasisFunction object
            C: center of nucleus
    '''
    v = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            v += a.norm[ia]*b.norm[ib]*ca*cb*nuclear_attraction(a.exps[ia],a.shell,a.origin,b.exps[ib],b.shell,b.origin,C)
    return -v*Z


def ERI(a,b,c,d):
    '''Evaluates overlap between two contracted Gaussians
            Returns float.
            Arguments:
            a: contracted Gaussian 'a', BasisFunction object
            b: contracted Gaussian 'b', BasisFunction object
            c: contracted Gaussian 'b', BasisFunction object
            d: contracted Gaussian 'b', BasisFunction object
    '''
    eri = 0.0
    for ja, ca in enumerate(a.coefs):
        for jb, cb in enumerate(b.coefs):
            for jc, cc in enumerate(c.coefs):
                for jd, cd in enumerate(d.coefs):
                    eri += a.norm[ja]*b.norm[jb]*c.norm[jc]*d.norm[jd]*\
                           ca*cb*cc*cd*\
                           electron_repulsion(a.exps[ja],a.shell,a.origin,\
                                b.exps[jb],b.shell,b.origin,\
                                c.exps[jc],c.shell,c.origin,\
                                d.exps[jd],d.shell,d.origin)
    return eri

In [7]:
#### General Functions ####


def construct_orbitals(SHELL, XI, CM, atom_coordinates):
    orbs = []; Nat = len(SHELL);
    for i in range(Nat):
        for j in range(len(XI[i])):
            orbs.append(BasisFunction(origin=atom_coordinates[i][:],shell=SHELL[i][j],exps=XI[i][j],coefs=CM[i][j]))
    return orbs


def ao_to_mo(AO,C):
    if len(AO.shape) == 2:
        T1 = np.einsum('jb,ij->ib',C,AO,optimize=True)
        T2 = np.einsum('ia,ib->ab',C,T1,optimize=True)
        return T2
    else:
        T1 = np.einsum('ld,ijkl->ijkd',C,AO,optimize=True)
        T2 = np.einsum('kc,ijkd->ijcd',C,T1,optimize=True)
        T3 = np.einsum('jb,ijcd->ibcd',C,T2,optimize=True)
        T4 = np.einsum('ia,ibcd->abcd',C,T3,optimize=True)
        return T4


def spatial_to_spinorb(MO):
    Norb = MO.shape[0]
    if len(MO.shape) == 2:
        MO_sp = np.zeros((2*Norb,2*Norb))
        for i in range(2*Norb):
            for j in range(2*Norb):
                if i%2 == j%2:    
                    i0 = int(np.floor(i/2))
                    j0 = int(np.floor(j/2))
                    MO_sp[i,j] = MO[i0,j0]
            
    else:
        MO_sp = np.zeros((2*Norb,2*Norb,2*Norb,2*Norb))
        for i in range(2*Norb):
            for j in range(2*Norb):
                for k in range(2*Norb):
                    for l in range(2*Norb):
                        if i%2 == k%2 and j%2 == l%2:
                            i0 = int(np.floor(i/2))
                            j0 = int(np.floor(j/2))
                            k0 = int(np.floor(k/2))
                            l0 = int(np.floor(l/2))
                            MO_sp[i,j,k,l] = MO[i0,j0,k0,l0]
    return MO_sp
 

def diis_pulay_solver(X_list,diis_resid_list):
        B_dim = len(X_list) + 1
        B = np.empty((B_dim, B_dim))
        B[-1, :] = -1
        B[:, -1] = -1
        B[-1, -1] = 0
        for i in range(len(X_list)):
            for j in range(i,len(X_list)):
                B[i, j] = np.einsum('ij,ij->', diis_resid_list[i], diis_resid_list[j], optimize=True)
                B[j, i] = B[i, j]

        # Build RHS of Pulay equation 
        rhs = np.zeros((B_dim))
        rhs[-1] = -1

        # Solve Pulay equation for c_i's 
        if np.linalg.det(B) != 0:
            coeff = np.linalg.solve(B, rhs)
            # Build DIIS Fock matrix 
            X = np.zeros_like(X_list[0])
            for x in range(B_dim - 1):
                X += coeff[x] * X_list[x]
            return X
        else:
            return X_list[-1]

In [8]:
#### SCF Functions ####

def calculate_onebody_ints(orbs,atom_coordinates):
    # Number of spatial orbitals
    Norb = len(orbs)
    # Allocate 
    Smat = np.zeros((Norb,Norb))
    Tmat = np.zeros((Norb,Norb))
    Vmat = np.zeros((Norb,Norb))
    # Calculate onebody integrals
    for i in range(Norb):
        for j in range(i,Norb):
            Smat[i,j] = S(orbs[i],orbs[j])
            Tmat[i,j] = T(orbs[i],orbs[j])
            for k in range(Nat):
                Vmat[i,j] += V(orbs[i],orbs[j],atom_coordinates[k][:],Z[k])
            Smat[j,i] = Smat[i,j]
            Tmat[j,i] = Tmat[i,j]
            Vmat[j,i] = Vmat[i,j]
    return Smat,Tmat,Vmat


def calculate_twobody_ints(orbs,vec):
    
    Norb = len(orbs)
    VVmat = np.zeros((Norb,Norb,Norb,Norb))
    
    if vec: # slower?
        num_twobody = int(Norb*(Norb+1)*(Norb**2+Norb+2)/8)
        VVmatvec = np.zeros(num_twobody)
        # Calculate only permutationally unique twobody integrals
        ct = 0
        for i in range(Norb):
            for j in range(i+1):
                for k in range(i+1):
                    if i == k: 
                        lmax = j+1 
                    else: 
                        lmax = k+1
                    for l in range(lmax):
                        VVmatvec[ct] = ERI(orbs[i],orbs[j],orbs[k],orbs[l]) # chemist notation
                        ct += 1
        # Insert unique integrals into 4-dimensional array
        for i in range(Norb):
            for j in range(Norb):
                if i > j: 
                    ij = i*(i+1)/2+j 
                else: 
                    ij = j*(j+1)/2 + i
                for k in range(Norb):
                    for l in range(Norb):
                        if k > l: 
                            kl = k*(k+1)/2+l 
                        else: 
                            kl = l*(l+1)/2 + k
                        if ij > kl: 
                            ijkl = ij*(ij+1)/2 + kl 
                        else: 
                            ijkl = kl*(kl+1)/2 + ij       
                        VVmat[i,k,j,l] = VVmatvec[int(ijkl)] # physics notation
        return VVmat,VVmatvec
    else:
        # Calculate twobody integrals
        for i in range(Norb):
            for j in range(i+1):
                ij = i*(i+1)/2 + j
                for k in range(Norb):
                    for l in range(k+1):
                        kl = k*(k+1)//2 + l
                        if ij >= kl:
                            val = ERI(orbs[i],orbs[j],orbs[k],orbs[l])
                            VVmat[i,k,j,l] = val
                            VVmat[j,k,i,l] = val
                            VVmat[i,l,j,k] = val
                            VVmat[k,i,l,j] = val
                            VVmat[l,i,k,j] = val
                            VVmat[l,j,k,i] = val
                            VVmat[j,l,i,k] = val
                            VVmat[k,j,l,i] = val
        return VVmat

def calc_nuclear_nuclear(atom_coordinates,Z):
    Vnn = 0.0
    Nat = len(atom_coordinates)
    for i in range(Nat):
        for j in range(Nat):
            if i != j:
                Ri = np.asarray(atom_coordinates[i][:])
                Rj = np.asarray(atom_coordinates[j][:])
                Vnn += Z[i]*Z[j]/np.linalg.norm(Ri-Rj)
    return Vnn/2

def orthomat(S,tol,kind):
    evalS, U = np.linalg.eigh(S)
    diagS_minushalf = np.diag(evalS**(-0.5))
    if kind == 'symmetric':
        X0 = np.dot(U,np.dot(diagS_minushalf,U.T))
    else:
        X0 = np.dot(U,diagS_minushalf)
        
    idX = [i for i in range(S.shape[0])]
    for i in range(S.shape[0]):
        for j in range(i+1,S.shape[0]):
            if np.abs(evalS[i]-evalS[j]) < tol:
                J = idX.index(j)
                idX.remove(J)
    X = X0[:,idX]
    return X

def print_MO(MOmat,unique,trim):
        
        Norb = MOmat.shape[0]
    
        if len(MOmat.shape) > 2:
            case = 'twobody'
        else:
            case = 'onebody'
            
        if trim == 'T' or trim == 1 or trim == 'True':
            if case == 'onebody':
                for i in range(Norb):
                    for j in range(Norb):
                        if np.abs(MOmat[i,j]) < 10**-10:
                            MOmat[i,j] = 0.0
            if case == 'twobody':
                for i in range(Norb):
                    for j in range(Norb):
                        for k in range(Norb):
                            for l in range(Norb):
                                if np.abs(MOmat[i,j,k,l]) < 10**-10:
                                    MOmat[i,j,k,l] = 0.0
            
        if case == 'onebody':
            ct = 0
            for i in range(Norb):
                for j in range(i+1):
                    ct += 1
                    print('{: >20}   {: >2}'.format(MOmat[i,j],ct))
                    
        if case == 'twobody':
            VV = np.einsum('pqrs->prqs',MOmat) # convert to chemist notation
            for i in range(Norb):
                for j in range(i+1):
                    for k in range(i+1):
                        if i == k: 
                            lmax = j+1 
                        else: 
                            lmax = k+1
                        for l in range(lmax):
                            print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[i,j,k,l],i+1,j+1,k+1,l+1))
                            if unique == 'False' or unique == 0 or unique == 'F':
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[i,j,l,k],i+1,j+1,l+1,k+1))
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[j,i,k,l],j+1,i+1,k+1,l+1))
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[j,i,l,k],j+1,i+1,l+1,k+1))
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[k,l,i,j],k+1,l+1,i+1,j+1))
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[l,k,i,j],l+1,k+1,i+1,j+1))
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[k,l,j,i],k+1,l+1,j+1,i+1))
                                print('{: >20}   {: >2}  {: >2}  {: >2}  {: >2}'.format(VV[l,k,j,i],l+1,k+1,j+1,i+1))
          
            
        return

In [9]:
#### CCSD Functions ####

def cc_energy(t1,t2,VM,FM,Nocc):
    Ecc = np.einsum('ia,ai->',FM[:Nocc,Nocc:],t1,optimize=True) +\
          0.25*np.einsum('ijab,abij->',VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True) +\
          0.5*np.einsum('jb,bj->',np.einsum('ijab,ai->jb',VM[:Nocc,:Nocc,Nocc:,Nocc:],t1,optimize=True),t1,optimize=True)
    return Ecc
                
def t1_fcn(t1,t2,VM,FM,Nocc,Nunocc):
    
    # Fock masks to zero diagonal
    Zocc = np.ones((Nocc,Nocc)) - np.eye(Nocc)
    Zunocc = np.ones((Nunocc,Nunocc)) - np.eye(Nunocc)
    
    # Intermediates
    chi_me = FM[:Nocc,Nocc:] + np.einsum('mnef,fn->me',VM[:Nocc,:Nocc,Nocc:,Nocc:],t1,optimize=True)
    
    chi_mi = FM[:Nocc,:Nocc]*Zocc + \
             np.einsum('mnif,fn->mi',VM[:Nocc,:Nocc,:Nocc,Nocc:],t1,optimize=True)+ \
             0.5*np.einsum('mnef,efin->mi',VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True)
    
    chi_ae = FM[Nocc:,Nocc:]*Zunocc + \
             np.einsum('anef,fn->ae',VM[Nocc:,:Nocc,Nocc:,Nocc:],t1,optimize=True) - \
             0.5*np.einsum('mnef,afmn->ae',VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True) - \
             np.einsum('me,am->ae',chi_me,t1,optimize=True)
    
    TEMP1 = FM[Nocc:,:Nocc] + \
            np.einsum('me,aeim->ai',chi_me,t2,optimize=True) - \
            np.einsum('maie,em->ai',VM[:Nocc,Nocc:,:Nocc,Nocc:],t1,optimize=True) -\
            np.einsum('mi,am->ai',chi_mi,t1,optimize=True) + \
            np.einsum('ae,ei->ai',chi_ae,t1,optimize=True) - \
            0.5*np.einsum('mnif,afmn->ai',VM[:Nocc,:Nocc,:Nocc,Nocc:],t2,optimize=True) + \
            0.5*np.einsum('anef,efin->ai',VM[Nocc:,:Nocc,Nocc:,Nocc:],t2,optimize=True)
    

    return TEMP1

def t2_fcn(t1,t2,VM,FM,Nocc,Nunocc):
    
    # Fock masks to zero diagonal
    Zocc = np.ones((Nocc,Nocc)) - np.eye(Nocc)
    Zunocc = np.ones((Nunocc,Nunocc)) - np.eye(Nunocc)
    
    # Intermediates    
    chi_mi = FM[:Nocc,:Nocc]*Zocc + \
             np.einsum('mnif,fn->mi',VM[:Nocc,:Nocc,:Nocc,Nocc:],t1,optimize=True)+ \
             np.einsum('mnef,efin->mi',0.5*VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True)
    
    chi_me = FM[:Nocc,Nocc:] + np.einsum('mnef,fn->me',VM[:Nocc,:Nocc,Nocc:,Nocc:],t1,optimize=True)
    
    chi_ae = FM[Nocc:,Nocc:]*Zunocc + \
             np.einsum('anef,fn->ae',VM[Nocc:,:Nocc,Nocc:,Nocc:],t1,optimize=True) - \
             0.5*np.einsum('mnef,afmn->ae',VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True) - \
             np.einsum('me,am->ae',chi_me,t1,optimize=True)
    
    chit_mi = chi_mi + np.einsum('me,ei->mi',chi_me,t1,optimize=True)
    
    chit_anef = VM[Nocc:,:Nocc,Nocc:,Nocc:] - \
                0.5*np.einsum('mnef,am->anef',VM[:Nocc,:Nocc,Nocc:,Nocc:],t1,optimize=True)
    
    chi_anej = VM[Nocc:,:Nocc,Nocc:,:Nocc] - \
               0.5*np.einsum('mnej,am->anej',VM[:Nocc,:Nocc,Nocc:,:Nocc],t1,optimize=True) + \
               0.5*np.einsum('anef,fj->anej',chit_anef,t1,optimize=True)
    
    chi_mnif = VM[:Nocc,:Nocc,:Nocc,Nocc:] + \
               0.5*np.einsum('mnef,ei->mnif',VM[:Nocc,:Nocc,Nocc:,Nocc:],t1,optimize=True)
    
    chi_mbij = VM[:Nocc,Nocc:,:Nocc,:Nocc] - \
                0.5*np.einsum('mnij,bn->mbij',VM[:Nocc,:Nocc,:Nocc,:Nocc],t1,optimize=True) - \
                0.5*np.einsum('bmef,efij->mbij',chit_anef,t2,optimize=True)
    
    chi_abej = 0.5*VM[Nocc:,Nocc:,Nocc:,:Nocc] - np.einsum('anej,bn->abej',chi_anej,t1,optimize=True)
    
    chi_mnij = 0.5*VM[:Nocc,:Nocc,:Nocc,:Nocc] + \
               0.25*np.einsum('mnef,efij->mnij',VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True) + \
               np.einsum('mnif,fj->mnij',chi_mnif,t1,optimize=True)
    
    chi_anef = chit_anef - 0.5*np.einsum('mnef,am->anef',VM[:Nocc,:Nocc,Nocc:,Nocc:],t1,optimize=True)
    
    chit_anej = VM[Nocc:,:Nocc,Nocc:,:Nocc] - \
                np.einsum('mnej,am->anej',VM[:Nocc,:Nocc,Nocc:,:Nocc],t1,optimize=True) - \
                0.5*np.einsum('mnef,afmj->anej',VM[:Nocc,:Nocc,Nocc:,Nocc:],t2,optimize=True) +\
                np.einsum('anef,fj->anej',chi_anef,t1,optimize=True)
    
    chi_efij = np.einsum('ei,fj->efij',t1,t1,optimize=True) + 0.5*t2
    
    # standard order
    TEMP2 = np.einsum('abej,ei->abij',chi_abej,t1,optimize=True) - \
            np.einsum('anej,ebin->abij',chit_anej,t2,optimize=True) - \
            0.5*np.einsum('mbij,am->abij',chi_mbij,t1,optimize=True) + \
            0.5*np.einsum('ae,ebij->abij',chi_ae,t2,optimize=True) - \
            0.5*np.einsum('mi,abmj->abij',chit_mi,t2,optimize=True) + \
            0.25*np.einsum('mnij,abmn->abij',chi_mnij,t2,optimize=True) +\
            0.25*np.einsum('abef,efij->abij',VM[Nocc:,Nocc:,Nocc:,Nocc:],chi_efij,optimize=True) 
    # (ij)
    TEMP2_ij = np.einsum('abei,ej->abij',chi_abej,t1,optimize=True) - \
               np.einsum('anei,ebjn->abij',chit_anej,t2,optimize=True) - \
               0.5*np.einsum('mbji,am->abij',chi_mbij,t1,optimize=True) + \
               0.5*np.einsum('ae,ebji->abij',chi_ae,t2,optimize=True) - \
            0.5*np.einsum('mj,abmi->abij',chit_mi,t2,optimize=True) + \
               0.25*np.einsum('mnji,abmn->abij',chi_mnij,t2,optimize=True) +\
            0.25*np.einsum('abef,efji->abij',VM[Nocc:,Nocc:,Nocc:,Nocc:],chi_efij,optimize=True) 
    # (ab)
    TEMP2_ab = np.einsum('baej,ei->abij',chi_abej,t1,optimize=True) - \
               np.einsum('bnej,eain->abij',chit_anej,t2,optimize=True) - \
               0.5*np.einsum('maij,bm->abij',chi_mbij,t1,optimize=True) + \
               0.5*np.einsum('be,eaij->abij',chi_ae,t2,optimize=True) - \
            0.5*np.einsum('mi,bamj->abij',chit_mi,t2,optimize=True) + \
               0.25*np.einsum('mnij,bamn->abij',chi_mnij,t2,optimize=True) +\
            0.25*np.einsum('baef,efij->abij',VM[Nocc:,Nocc:,Nocc:,Nocc:],chi_efij,optimize=True)
    # (ij)(ab)
    TEMP2_abij = np.einsum('baei,ej->abij',chi_abej,t1,optimize=True) - \
                 np.einsum('bnei,eajn->abij',chit_anej,t2,optimize=True) - \
                 0.5*np.einsum('maji,bm->abij',chi_mbij,t1,optimize=True) + \
                 0.5*np.einsum('be,eaji->abij',chi_ae,t2,optimize=True) - \
                 0.5*np.einsum('mj,bami->abij',chit_mi,t2,optimize=True) + \
                0.25*np.einsum('mnji,bamn->abij',chi_mnij,t2,optimize=True) +\
                 0.25*np.einsum('baef,efji->abij',VM[Nocc:,Nocc:,Nocc:,Nocc:],chi_efij,optimize=True)  
    
    return VM[Nocc:,Nocc:,:Nocc,:Nocc] + TEMP2 - TEMP2_ij - TEMP2_ab + TEMP2_abij

def t1_update(t1,t2,VM,FM,occ,unocc):
    Nocc = len(occ)
    Nunocc = len(unocc)
    X_ai = t1_fcn(t1,t2,VM,FM,Nocc,Nunocc)
    for a in range(Nunocc):
        for i in range(Nocc):
            D_ai = FM[occ[i],occ[i]]-FM[unocc[a],unocc[a]]
            t1[a,i] = X_ai[a,i]/D_ai
    return t1

def t2_update(t1,t2,VM,FM,occ,unocc):
    Nocc = len(occ)
    Nunocc = len(unocc)
    X_abij = t2_fcn(t1,t2,VM,FM,Nocc,Nunocc)
    for a in range(Nunocc):
        for b in range(a+1,Nunocc):
            for i in range(Nocc):
                for j in range(i+1,Nocc):
                    D2_abij = FM[occ[i],occ[i]]+FM[occ[j],occ[j]]-FM[unocc[a],unocc[a]]-FM[unocc[b],unocc[b]]
                    t2[a,b,i,j] = X_abij[a,b,i,j]/D2_abij
                    t2[b,a,i,j] = -t2[a,b,i,j]
                    t2[a,b,j,i] = -t2[a,b,i,j]
                    t2[b,a,j,i] = t2[a,b,i,j]
    return t2


In [10]:
#### Left-CCSD Functions ####

def lambda1_fcn(lambda1,lambda2,t1,t2,VM,FM,Nocc,Nunocc):
    
    fvo = FM[Nocc:,:Nocc]
    fov = FM[:Nocc,Nocc:]
    foo = FM[:Nocc,:Nocc]
    fvv = FM[Nocc:,Nocc:]
    
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    Vooov = VM[:Nocc,:Nocc,:Nocc,Nocc:]
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Vovov = VM[:Nocc,Nocc:,:Nocc,Nocc:]
    Vvoov = VM[Nocc:,:Nocc,:Nocc,Nocc:]
    Vvovo = VM[Nocc:,:Nocc,Nocc:,:Nocc]
    Voovo = VM[:Nocc,:Nocc,Nocc:,:Nocc]
    Vvovv = VM[Nocc:,:Nocc,Nocc:,Nocc:]
    Vvvvv = VM[Nocc:,Nocc:,Nocc:,Nocc:]
    Vvooo = VM[Nocc:,:Nocc,:Nocc,:Nocc]
    Voooo = VM[:Nocc,:Nocc,:Nocc,:Nocc]
    Vvvov = VM[Nocc:,Nocc:,:Nocc,Nocc:]
    Vvvvo = VM[Nocc:,Nocc:,Nocc:,:Nocc]
    Vovoo = VM[:Nocc,Nocc:,:Nocc,:Nocc]
    Vovvo = VM[:Nocc,Nocc:,Nocc:,:Nocc]
    
    
    # Fock masks to zero diagonal
    Zocc = np.ones((Nocc,Nocc)) - np.eye(Nocc)
    Zunocc = np.ones((Nunocc,Nunocc)) - np.eye(Nunocc)
    
    # Intermediates
    tau = t2 + 0.5*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                    np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                    np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                    np.einsum('bj,ai->abij',t1,t1,optimize=True)) 
    
    taubar = t2 + 0.25*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                        np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                        np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                        np.einsum('bj,ai->abij',t1,t1,optimize=True)) 

    Fbar_ae =   fvv - \
                0.5*np.einsum('me,am->ae',fov,t1,optimize=True) + \
                np.einsum('amef,fm->ae',Vvovv,t1,optimize=True) - \
                0.5*np.einsum('afmn,mnef->ae',taubar,Voovv,optimize=True)
                
    Fbar_mi = foo + \
              0.5*np.einsum('me,ei->mi',fov,t1,optimize=True)  + \
              np.einsum('mnie,en->mi',Vooov,t1,optimize=True) + \
              0.5*np.einsum('mnef,efin->mi',Voovv,taubar,optimize=True)
                
    Fbar_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
            
    Wbar_mbej = Vovvo + \
                np.einsum('mbef,fj->mbej',Vovvv,t1,optimize=True) - \
                np.einsum('mnej,bn->mbej',Voovo,t1,optimize=True) - \
                0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True) - \
                np.einsum('mnej,bn->mbej',np.einsum('mnef,fj->mnej',Voovv,t1,optimize=True),t1,optimize=True)
    
    # Hbar 1-body matrix elements
    F_ae = Fbar_ae - 0.5*np.einsum('me,am->ae',Fbar_me,t1,optimize=True)
    
    F_mi = Fbar_mi + 0.5*np.einsum('me,ei->mi',Fbar_me,t1,optimize=True)
    
    F_me = Fbar_me
    
    # Hbar 2-body matrix elements
    W_mbej = Wbar_mbej - 0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True)
    
    W_efam = Vvvvo + np.einsum('efag,gm->efam',Vvvvv,t1,optimize=True) -\
             0.5*np.einsum('efno,noam->efam',tau,Voovo+np.einsum('noag,gm->noam',Voovv,t1,optimize=True),optimize=True) - \
             np.einsum('enag,fgnm->efam',Vvovv,taubar,optimize=True) + \
             np.einsum('fnag,egnm->efam',Vvovv,taubar,optimize=True) + \
             np.einsum('en,fnam->efam',t1,Vvovo-np.einsum('noag,fgmo->fnam',Voovv,t2,optimize=True),optimize=True) - \
             np.einsum('fn,enam->efam',t1,Vvovo-np.einsum('noag,egmo->enam',Voovv,t2,optimize=True),optimize=True) - \
             np.einsum('efnm,ma->efam',t2,F_me,optimize=True)
    
    W_iemn = Vovoo - np.einsum('iomn,eo->iemn',Voooo,t1,optimize=True) + \
             0.5*np.einsum('fgmn,iefg->iemn',tau,Vovvv-np.einsum('iofg,eo->iefg',Voovv,t1,optimize=True),optimize=True) + \
             np.einsum('iomf,efno->iemn',Vooov,taubar,optimize=True) - \
             np.einsum('ionf,efmo->iemn',Vooov,taubar,optimize=True) + \
             np.einsum('fn,iemf->iemn',t1,Vovov-np.einsum('iofg,egmo->iemf',Voovv,t2,optimize=True),optimize=True) - \
             np.einsum('fm,ienf->iemn',t1,Vovov-np.einsum('iofg,egno->ienf',Voovv,t2,optimize=True),optimize=True) + \
             np.einsum('efnm,if->iemn',t2,F_me,optimize=True)

    
    # Hbar 3-body matrix elements
    W_abmije =  -np.einsum('mnej,abin->abmije',Voovo,t2,optimize=True) + np.einsum('mnei,abjn->abmije',Voovo,t2,optimize=True) -\
                np.einsum('mnei,abnj->abmije',np.einsum('mnef,fi->mnei',Voovv,t1,optimize=True),t2,optimize=True) + \
                np.einsum('mnej,abni->abmije',np.einsum('mnef,fj->mnej',Voovv,t1,optimize=True),t2,optimize=True) + \
                np.einsum('mbef,afij->abmije',Vovvv,t2,optimize=True) - np.einsum('maef,bfij->abmije',Vovvv,t2,optimize=True) - \
                np.einsum('maef,fbij->abmije',np.einsum('mnef,an->maef',Voovv,t1,optimize=True),t2,optimize=True) + \
                np.einsum('mbef,faij->abmije',np.einsum('mnef,bn->mbef',Voovv,t1,optimize=True),t2,optimize=True)
    
    
    TEMP1 = ( F_me + np.einsum('ae,ma->me',F_ae*Zunocc,lambda1,optimize=True) -\
                      np.einsum('mi,ma->ia',F_mi*Zocc,lambda1,optimize=True)  + \
                      0.5*np.einsum('efam,imef->ia',W_efam,lambda2,optimize=True) + \
                      np.einsum('ieam,me->ie',W_mbej,lambda1,optimize=True) - \
                      0.5*np.einsum('iemn,mnae->ia',W_iemn,lambda2,optimize=True) + \
                      0.25*np.einsum('abmije,ijab->me',W_abmije,lambda2,optimize=True) )


    return TEMP1


def lambda2_fcn(lambda1,lambda2,t1,t2,VM,FM,Nocc,Nunocc):
    
    fvo = FM[Nocc:,:Nocc]
    fov = FM[:Nocc,Nocc:]
    foo = FM[:Nocc,:Nocc]
    fvv = FM[Nocc:,Nocc:]
    
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    Vooov = VM[:Nocc,:Nocc,:Nocc,Nocc:]
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Vovov = VM[:Nocc,Nocc:,:Nocc,Nocc:]
    Vvoov = VM[Nocc:,:Nocc,:Nocc,Nocc:]
    Vvovo = VM[Nocc:,:Nocc,Nocc:,:Nocc]
    Voovo = VM[:Nocc,:Nocc,Nocc:,:Nocc]
    Vvovv = VM[Nocc:,:Nocc,Nocc:,Nocc:]
    Vvvvv = VM[Nocc:,Nocc:,Nocc:,Nocc:]
    Vvooo = VM[Nocc:,:Nocc,:Nocc,:Nocc]
    Voooo = VM[:Nocc,:Nocc,:Nocc,:Nocc]
    Vvvov = VM[Nocc:,Nocc:,:Nocc,Nocc:]
    Vvvvo = VM[Nocc:,Nocc:,Nocc:,:Nocc]
    Vovoo = VM[:Nocc,Nocc:,:Nocc,:Nocc]
    Vovvo = VM[:Nocc,Nocc:,Nocc:,:Nocc]
    
    # Fock masks to zero diagonal
    Zocc = np.ones((Nocc,Nocc)) - np.eye(Nocc)
    Zunocc = np.ones((Nunocc,Nunocc)) - np.eye(Nunocc)
    
    # Matrix elements and equations by my derivation, confirmed with
    # Gauss & Stanton, J. Chem. Phys. 103, 3561 (1995)
    
    # Intermediates
    tau = t2 + 0.5*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                    np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                    np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                    np.einsum('bj,ai->abij',t1,t1,optimize=True)) 
    
    taubar = t2 + 0.25*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                        np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                        np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                        np.einsum('bj,ai->abij',t1,t1,optimize=True)) 

    Fbar_ae =   fvv - \
                0.5*np.einsum('me,am->ae',fov,t1,optimize=True) + \
                np.einsum('amef,fm->ae',Vvovv,t1,optimize=True) - \
                0.5*np.einsum('afmn,mnef->ae',taubar,Voovv,optimize=True)
                
    Fbar_mi = foo + \
              0.5*np.einsum('me,ei->mi',fov,t1,optimize=True)  + \
              np.einsum('mnie,en->mi',Vooov,t1,optimize=True) + \
              0.5*np.einsum('mnef,efin->mi',Voovv,taubar,optimize=True)
                
    Fbar_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
    
    Wbar_mnij = Voooo + \
                np.einsum('mnie,ej->mnij',Vooov,t1,optimize=True) - np.einsum('mnje,ei->mnij',Vooov,t1,optimize=True) + \
                0.25*np.einsum('mnef,efij->mnij',Voovv,tau,optimize=True)
        
    Wbar_abef = Vvvvv -\
                np.einsum('amef,bm->abef',Vvovv,t1,optimize=True) + np.einsum('bmef,am->abef',Vvovv,t1,optimize=True) + \
                0.25*np.einsum('mnef,abmn->abef',Voovv,tau,optimize=True)
            
    Wbar_mbej = Vovvo + \
                np.einsum('mbef,fj->mbej',Vovvv,t1,optimize=True) - \
                np.einsum('mnej,bn->mbej',Voovo,t1,optimize=True) - \
                0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True) - \
                np.einsum('mnej,bn->mbej',np.einsum('mnef,fj->mnej',Voovv,t1,optimize=True),t1,optimize=True)
    
    # Hbar 1-body matrix elements
    F_ae = Fbar_ae - 0.5*np.einsum('me,am->ae',Fbar_me,t1,optimize=True)
    
    F_mi = Fbar_mi + 0.5*np.einsum('me,ei->mi',Fbar_me,t1,optimize=True)
    
    F_me = Fbar_me
    
    # Hbar 2-body matrix elements
    W_mnij = Wbar_mnij + 0.25*np.einsum('mnef,efij->mnij',Voovv,tau,optimize=True)
    
    W_abef = Wbar_abef + 0.25*np.einsum('mnef,abmn->abef',Voovv,tau,optimize=True)
    
    W_mbej = Wbar_mbej - 0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True)
    
    W_mnie = Vooov + np.einsum('mnfe,fi->mnie',Voovv,t1,optimize=True)
    
    W_amef = Vvovv - np.einsum('nmef,an->amef',Voovv,t1,optimize=True)

    # Hbar 3-body matrix elements
    W_amnije = np.einsum('mnfe,afij->amnije',Voovv,t2,optimize=True)
    
    W_abmief = -np.einsum('mnfe,abin->abmief',Voovv,t2,optimize=True)
    
    
    D1 = Voovv
    D2 = np.einsum('ejab,ie->ijab',W_amef,lambda1,optimize=True) - np.einsum('eiab,je->ijab',W_amef,lambda1,optimize=True)
    D3 = -np.einsum('ijmb,ma->ijab',W_mnie,lambda1,optimize=True) + np.einsum('ijma,mb->ijab',W_mnie,lambda1,optimize=True)
    D4 = 0.5*np.einsum('efab,ijef->ijab',W_abef,lambda2,optimize=True)
    D5 = 0.5*np.einsum('ijmn,mnab->ijab',W_mnij,lambda2,optimize=True)
    D6 = np.einsum('jebm,imae->ijab',W_mbej,lambda2,optimize=True) - \
         np.einsum('jeam,imbe->ijab',W_mbej,lambda2,optimize=True) - \
         np.einsum('iebm,jmae->ijab',W_mbej,lambda2,optimize=True) + \
         np.einsum('ieam,jmbe->ijab',W_mbej,lambda2,optimize=True)
    D7 = 0.5*np.einsum('fejmab,imef->ijab',W_abmief,lambda2,optimize=True) - \
         0.5*np.einsum('feimab,jmef->ijab',W_abmief,lambda2,optimize=True)
    D8 = -0.5*np.einsum('eijnmb,mnae->ijab',W_amnije,lambda2,optimize=True) + \
          0.5*np.einsum('eijnma,mnbe->ijab',W_amnije,lambda2,optimize=True)
    D9 = np.einsum('ea,ijeb->ijab',F_ae*Zunocc,lambda2,optimize=True) - \
         np.einsum('eb,ijea->ijab',F_ae*Zunocc,lambda2,optimize=True)
    D10 = -np.einsum('im,mjab->ijab',F_mi*Zocc,lambda2,optimize=True) + \
           np.einsum('jm,miab->ijab',F_mi*Zocc,lambda2,optimize=True)
    D11 = np.einsum('ia,jb->ijab',lambda1,F_me,optimize=True) - \
          np.einsum('ja,ib->ijab',lambda1,F_me,optimize=True) - \
          np.einsum('ib,ja->ijab',lambda1,F_me,optimize=True) + \
          np.einsum('jb,ia->ijab',lambda1,F_me,optimize=True)
    
    TEMP2 = (D1 + D2 + D3 + D4 + D5 + D6 + D7 + D8 + D9 + D10 + D11)
    
    return TEMP2
    
def lambda1_update(lambda1,lambda2,t1,t2,VM,FM,occ,unocc):
    
    Nocc = len(occ)
    Nunocc = len(unocc)
    
    foo = FM[:Nocc,:Nocc]
    fvv = FM[Nocc:,Nocc:]
    fov = FM[:Nocc,Nocc:]
    Vvovv = VM[Nocc:,:Nocc,Nocc:,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    Vooov = VM[:Nocc,:Nocc,:Nocc,Nocc:]
    
    taubar = t2 + 0.25*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                    np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                    np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                    np.einsum('bj,ai->abij',t1,t1,optimize=True)) 

    Fbar_ae =   fvv - \
                0.5*np.einsum('me,am->ae',fov,t1,optimize=True) + \
                np.einsum('amef,fm->ae',Vvovv,t1,optimize=True) - \
                0.5*np.einsum('afmn,mnef->ae',taubar,Voovv,optimize=True)
                
    Fbar_mi = foo + \
              0.5*np.einsum('me,ei->mi',fov,t1,optimize=True)  + \
              np.einsum('mnie,en->mi',Vooov,t1,optimize=True) + \
              0.5*np.einsum('mnef,efin->mi',Voovv,taubar,optimize=True)
    
    Fbar_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
    
    F_ae = Fbar_ae - 0.5*np.einsum('me,am->ae',Fbar_me,t1,optimize=True)
    
    F_mi = Fbar_mi + 0.5*np.einsum('me,ei->mi',Fbar_me,t1,optimize=True)
    
    X_ia = lambda1_fcn(lambda1,lambda2,t1,t2,VM,FM,Nocc,Nunocc)
    for i in range(Nocc):
        for a in range(Nunocc):
            D_ai = F_mi[i,i]-F_ae[a,a]
            lambda1[i,a] = X_ia[i,a]/D_ai
    return lambda1

def lambda2_update(lambda1,lambda2,t1,t2,VM,FM,occ,unocc):
    
    Nocc = len(occ)
    Nunocc = len(unocc)
    
    foo = FM[:Nocc,:Nocc]
    fvv = FM[Nocc:,Nocc:]
    fov = FM[:Nocc,Nocc:]
    Vvovv = VM[Nocc:,:Nocc,Nocc:,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    Vooov = VM[:Nocc,:Nocc,:Nocc,Nocc:]
    
    taubar = t2 + 0.25*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                    np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                    np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                    np.einsum('bj,ai->abij',t1,t1,optimize=True)) 

    Fbar_ae =   fvv - \
                0.5*np.einsum('me,am->ae',fov,t1,optimize=True) + \
                np.einsum('amef,fm->ae',Vvovv,t1,optimize=True) - \
                0.5*np.einsum('afmn,mnef->ae',taubar,Voovv,optimize=True)
                
    Fbar_mi = foo + \
              0.5*np.einsum('me,ei->mi',fov,t1,optimize=True)  + \
              np.einsum('mnie,en->mi',Vooov,t1,optimize=True) + \
              0.5*np.einsum('mnef,efin->mi',Voovv,taubar,optimize=True)
    
    Fbar_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
    
    F_ae = Fbar_ae - 0.5*np.einsum('me,am->ae',Fbar_me,t1,optimize=True)
    
    F_mi = Fbar_mi + 0.5*np.einsum('me,ei->mi',Fbar_me,t1,optimize=True)
    
    X_ijab = lambda2_fcn(lambda1,lambda2,t1,t2,VM,FM,Nocc,Nunocc)
    for i in range(Nocc):
        for j in range(i+1,Nocc):
            for a in range(Nunocc):
                for b in range(a+1,Nunocc):
                    D2_abij = F_mi[i,i]+F_mi[j,j]-F_ae[a,a]-F_ae[b,b]
                    lambda2[i,j,a,b] = X_ijab[i,j,a,b]/D2_abij
                    lambda2[j,i,a,b] = -lambda2[i,j,a,b]
                    lambda2[i,j,b,a] = -lambda2[i,j,a,b]
                    lambda2[j,i,b,a] = lambda2[i,j,a,b]
    return lambda2

In [11]:
#### EOM-CCSD Functions ####

def gramschmidt(A):
    """ Gram-Schmidt orthogonalization of column-vectors. Matrix A passes
    vectors in its columns, orthonormal system is returned in columns of
    matrix Q. """
    _, k = A.shape

    # first basis vector
    Q = A[:, [0]] / np.linalg.norm(A[:, 0])
    for j in range(1, k):
        # orthogonal projection, loop-free implementation
        q = A[:, j] - np.dot(Q, np.dot(Q.T, A[:, j]))

        # check premature termination
        nq = np.linalg.norm(q)
        if nq < 1e-9 * np.linalg.norm(A[:, j]):
            break
        # add new basis vector as another column of Q
        Q = np.column_stack([Q, q / nq])
    return Q


def cis_hamiltonian(FM,VM,H00,occ,unocc):
    
    Nocc = len(occ)
    Nunocc = len(unocc)
    Nov = Nocc*Nunocc
    
    def swapPositions(s0, pos1, pos2): 
        s02 = s0.copy()
        s02[pos1] = s0[pos2]
        s02[pos2] = s0[pos1]
        return s02

    def singleExcList(s0,iocc,iunocc):
        singleExc = []
        singleExcIdx = []
        for i in iocc:
            for j in iunocc:
                singleExc.append(swapPositions(s0,i,j))
                singleExcIdx.append([i, j])
        return singleExc, singleExcIdx
    
    # singles excitation determinant list
    s0 = np.concatenate( (np.ones_like(occ),np.zeros_like(unocc)), axis=0)
    singles, singleIdx = singleExcList(s0,occ,unocc)
    
    # reference-reference block
    H00 = np.array([H00])[:,np.newaxis] # Hartree-Fock energy
    
    # singles-reference block
    HS0 = np.zeros(Nov)[:,np.newaxis]
    one_body = 0
    for a in range(len(singles)):
        D1 = singles[a]
        I = singleIdx[a][0]
        A = singleIdx[a][1]
        HS0[a,0] += FM[A,I]
            
    # singles-singles block
    HSS = np.zeros((Nov,Nov))
    for a in range(len(singles)):
        D1 = singles[a]
        I = singleIdx[a][0]
        A = singleIdx[a][1]
        for b in range(len(singles)):
            D2 = singles[b]
            J = singleIdx[b][0]
            B = singleIdx[b][1]
            HSS[a,b] = FM[A,B]*(I==J) - FM[I,J]*(A==B) + VM[A,J,I,B]
            
                       
    H_CIS = np.concatenate((np.concatenate((H00,HS0.T),axis=1),np.concatenate((HS0,HSS),axis=1)),axis=0)
    
    #H_CIS = HSS
        
    return H_CIS

def cis_guess(H_CIS,Nocc,Nunocc):
    
    # Diagonalize Hermitian CIS matrix
    omega,V = np.linalg.eigh(H_CIS)
    
    # Sort eigpairs
    idx = omega.argsort()
    omega = omega[idx]
    V = V[:,idx]
    for i in range(V.shape[1]):
        V[:,i] = V[:,i]/np.linalg.norm(V[:,i])
    
    # Discard singles-corrected ground state eigpair
    omega = omega[1:]
    V = V[1:,1:]

    return omega,V


def Hbar_diagonal(t1,t2,VM,FM,occ,unocc):
    
    Nocc = len(occ)
    Nunocc = len(unocc)
    
    foo = FM[:Nocc,:Nocc]
    fvv = FM[Nocc:,Nocc:]
    fov = FM[:Nocc,Nocc:]
    Vvovv = VM[Nocc:,:Nocc,Nocc:,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    Vooov = VM[:Nocc,:Nocc,:Nocc,Nocc:]
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Voovo = VM[:Nocc,:Nocc,Nocc:,:Nocc]
    Vovvo = VM[:Nocc,Nocc:,Nocc:,:Nocc]
    Vvvvv = VM[Nocc:,Nocc:,Nocc:,Nocc:]
    Voooo = VM[:Nocc,:Nocc,:Nocc,:Nocc]
    
    taubar = t2 + 0.25*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                        np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                        np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                        np.einsum('bj,ai->abij',t1,t1,optimize=True)) 
    
    tau = t2 + 0.5*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                np.einsum('bj,ai->abij',t1,t1,optimize=True)) 

    Fbar_ae =   fvv - \
                0.5*np.einsum('me,am->ae',fov,t1,optimize=True) + \
                np.einsum('amef,fm->ae',Vvovv,t1,optimize=True) - \
                0.5*np.einsum('afmn,mnef->ae',taubar,Voovv,optimize=True)
                
    Fbar_mi = foo + \
              0.5*np.einsum('me,ei->mi',fov,t1,optimize=True)  + \
              np.einsum('mnie,en->mi',Vooov,t1,optimize=True) + \
              0.5*np.einsum('mnef,efin->mi',Voovv,taubar,optimize=True)
                
    Fbar_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
    
    Wbar_mbej = Vovvo + \
            np.einsum('mbef,fj->mbej',Vovvv,t1,optimize=True) - \
            np.einsum('mnej,bn->mbej',Voovo,t1,optimize=True) - \
            0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True) - \
            np.einsum('mnej,bn->mbej',np.einsum('mnef,fj->mnej',Voovv,t1,optimize=True),t1,optimize=True)
    
    F_ae = Fbar_ae - 0.5*np.einsum('me,am->ae',Fbar_me,t1,optimize=True)
    
    F_mi = Fbar_mi + 0.5*np.einsum('me,ei->mi',Fbar_me,t1,optimize=True)
    
    W_mbej = Wbar_mbej - 0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True)
    
    Wbar_mnij = Voooo + \
                np.einsum('mnie,ej->mnij',Vooov,t1,optimize=True) - np.einsum('mnje,ei->mnij',Vooov,t1,optimize=True) + \
                0.25*np.einsum('mnef,efij->mnij',Voovv,tau,optimize=True)
        
    Wbar_abef = Vvvvv -\
                np.einsum('amef,bm->abef',Vvovv,t1,optimize=True) + np.einsum('bmef,am->abef',Vvovv,t1,optimize=True) + \
                0.25*np.einsum('mnef,abmn->abef',Voovv,tau,optimize=True)
            
    W_mnij = Wbar_mnij + 0.25*np.einsum('mnef,efij->mnij',Voovv,tau,optimize=True)
    
    W_abef = Wbar_abef + 0.25*np.einsum('mnef,abmn->abef',Voovv,tau,optimize=True)
    
    Dia = np.zeros((Nunocc,Nocc))
    Dijab = np.zeros((Nunocc,Nunocc,Nocc,Nocc))
    
    for a in range(Nunocc):
        for i in range(Nocc):
            
            Dia[a,i] += F_ae[a,a] - F_mi[i,i]
            
            # WHY IS THIS A MINUS???
            Dia[a,i] -= W_mbej[i,a,a,i]
            
            for b in range(Nunocc):
                for j in range(Nocc):
                    
                    # these terms come in paris of A(ab) and A(ij) but the sign is +1 for each antisymmetrizer
                    # because determinants on both the left and right flip sign
                    Dijab[a,b,i,j] += F_ae[a,a] + F_ae[b,b] - F_mi[i,i] - F_mi[j,j]
                    
                    # note these 4 terms come from A(ij)A(ab) but the signs are all +1 because both determinants in 
                    # <ijab|Hbar|ijab> get flipped so the antisymmetrizer sign is always +1
                    # WHY IS THIS A MINUS???
                    # Must be to do with the inherent sign of W_mbej... there is no funny business here!
                    Dijab[a,b,i,j] -= W_mbej[i,b,b,i] + W_mbej[j,b,b,j] + W_mbej[i,a,a,i] + W_mbej[j,a,a,j] 
                    
                    Dijab[a,b,i,j] += W_abef[a,b,a,b] + W_mnij[i,j,i,j]
                    
                    for l in range(Nocc):
                        Dijab[a,b,i,j] -= 0.5*(VM[occ[j],occ[l],unocc[a],unocc[b]]*t2[a,b,j,l] + \
                                               VM[occ[i],occ[l],unocc[a],unocc[b]]*t2[a,b,i,l])
                    for d in range(Nunocc):
                        Dijab[a,b,i,j] -= 0.5*(VM[occ[i],occ[j],unocc[b],unocc[d]]*t2[b,d,i,j] + \
                                               VM[occ[i],occ[j],unocc[a],unocc[d]]*t2[a,d,i,j])
    
                    
    D = np.hstack(( Dia.flatten(), Dijab.flatten()))
    idx = np.argsort(D)
    return D
    
    
def sigma_fcn(r1,r2,t1,t2,VM,FM,Nocc,Nunocc):
    
    fvo = FM[Nocc:,:Nocc]
    fov = FM[:Nocc,Nocc:]
    foo = FM[:Nocc,:Nocc]
    fvv = FM[Nocc:,Nocc:]
    
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    Vooov = VM[:Nocc,:Nocc,:Nocc,Nocc:]
    Vovvv = VM[:Nocc,Nocc:,Nocc:,Nocc:]
    Vovov = VM[:Nocc,Nocc:,:Nocc,Nocc:]
    Vvoov = VM[Nocc:,:Nocc,:Nocc,Nocc:]
    Vvovo = VM[Nocc:,:Nocc,Nocc:,:Nocc]
    Voovo = VM[:Nocc,:Nocc,Nocc:,:Nocc]
    Vvovv = VM[Nocc:,:Nocc,Nocc:,Nocc:]
    Vvvvv = VM[Nocc:,Nocc:,Nocc:,Nocc:]
    Vvooo = VM[Nocc:,:Nocc,:Nocc,:Nocc]
    Voooo = VM[:Nocc,:Nocc,:Nocc,:Nocc]
    Vvvov = VM[Nocc:,Nocc:,:Nocc,Nocc:]
    Vvvvo = VM[Nocc:,Nocc:,Nocc:,:Nocc]
    Vovoo = VM[:Nocc,Nocc:,:Nocc,:Nocc]
    Vovvo = VM[:Nocc,Nocc:,Nocc:,:Nocc]
    
    # Matrix elements and equations by my derivation, confirmed with
    # Gauss & Stanton, J. Chem. Phys. 103, 3561 (1995)
    
    # Intermediates
    tau = t2 + 0.5*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                    np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                    np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                    np.einsum('bj,ai->abij',t1,t1,optimize=True)) 
    
    taubar = t2 + 0.25*(np.einsum('ai,bj->abij',t1,t1,optimize=True) - \
                        np.einsum('aj,bi->abij',t1,t1,optimize=True) - \
                        np.einsum('bi,aj->abij',t1,t1,optimize=True) + \
                        np.einsum('bj,ai->abij',t1,t1,optimize=True)) 

    Fbar_ae =   fvv - \
                0.5*np.einsum('me,am->ae',fov,t1,optimize=True) + \
                np.einsum('amef,fm->ae',Vvovv,t1,optimize=True) - \
                0.5*np.einsum('afmn,mnef->ae',taubar,Voovv,optimize=True)
                
    Fbar_mi = foo + \
              0.5*np.einsum('me,ei->mi',fov,t1,optimize=True)  + \
              np.einsum('mnie,en->mi',Vooov,t1,optimize=True) + \
              0.5*np.einsum('mnef,efin->mi',Voovv,taubar,optimize=True)
                
    Fbar_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
    
    Wbar_mnij = Voooo + \
                np.einsum('mnie,ej->mnij',Vooov,t1,optimize=True) - np.einsum('mnje,ei->mnij',Vooov,t1,optimize=True) + \
                0.25*np.einsum('mnef,efij->mnij',Voovv,tau,optimize=True)
        
    Wbar_abef = Vvvvv -\
                np.einsum('amef,bm->abef',Vvovv,t1,optimize=True) + np.einsum('bmef,am->abef',Vvovv,t1,optimize=True) + \
                0.25*np.einsum('mnef,abmn->abef',Voovv,tau,optimize=True)
            
    Wbar_mbej = Vovvo + \
                np.einsum('mbef,fj->mbej',Vovvv,t1,optimize=True) - \
                np.einsum('mnej,bn->mbej',Voovo,t1,optimize=True) - \
                0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True) - \
                np.einsum('mnej,bn->mbej',np.einsum('mnef,fj->mnej',Voovv,t1,optimize=True),t1,optimize=True)
    
    # Hbar 1-body matrix elements
    F_ae = Fbar_ae - 0.5*np.einsum('me,am->ae',Fbar_me,t1,optimize=True)
    
    F_mi = Fbar_mi + 0.5*np.einsum('me,ei->mi',Fbar_me,t1,optimize=True)
    
    F_me = Fbar_me
    
    # Hbar 2-body matrix elements
    W_mnij = Wbar_mnij + 0.25*np.einsum('mnef,efij->mnij',Voovv,tau,optimize=True)
    
    W_abef = Wbar_abef + 0.25*np.einsum('mnef,abmn->abef',Voovv,tau,optimize=True)
    
    W_mbej = Wbar_mbej - 0.5*np.einsum('mnef,fbjn->mbej',Voovv,t2,optimize=True)
    
    W_mnie = Vooov + np.einsum('mnfe,fi->mnie',Voovv,t1,optimize=True)
    
    W_amef = Vvovv - np.einsum('nmef,an->amef',Voovv,t1,optimize=True)
    
    W_mbij =    Vovoo - np.einsum('me,beij->mbij',F_me,t2,optimize=True) - np.einsum('mnij,bn->mbij',W_mnij,t1,optimize=True) + \
                0.5*np.einsum('mbef,efij->mbij',Vovvv,tau,optimize=True) + \
                np.einsum('mnie,bejn->mbij',Vooov,t2,optimize=True) - np.einsum('mnje,bein->mbij',Vooov,t2,optimize=True) + \
                np.einsum('mbej,ei->mbij',Vovvo,t1,optimize=True) - np.einsum('mbei,ej->mbij',Vovvo,t1,optimize=True) - \
                np.einsum('mbej,ei->mbij',np.einsum('mnef,bfnj->mbej',Voovv,t2,optimize=True),t1,optimize=True) + \
                np.einsum('mbei,ej->mbij',np.einsum('mnef,bfni->mbei',Voovv,t2,optimize=True),t1,optimize=True)
                
    W_abei =    Vvvvo - np.einsum('me,abmi->abei',F_me,t2,optimize=True) + np.einsum('abef,fi->abei',W_abef,t1,optimize=True) + \
                0.5*np.einsum('mnei,abmn->abei',Voovo,tau,optimize=True) - \
                np.einsum('mbef,afmi->abei',Vovvv,t2,optimize=True) + np.einsum('maef,bfmi->abei',Vovvv,t2,optimize=True) - \
                np.einsum('mbei,am->abei',Vovvo,t1,optimize=True) + np.einsum('maei,bm->abei',Vovvo,t1,optimize=True) + \
                np.einsum('mbei,am->abei',np.einsum('mnef,bfni->mbei',Voovv,t2,optimize=True),t1,optimize=True) - \
                np.einsum('maei,bm->abei',np.einsum('mnef,afni->maei',Voovv,t2,optimize=True),t1,optimize=True)
    
    # Hbar 3-body matrix elements
    W_abmije =  -np.einsum('mnej,abin->abmije',Voovo,t2,optimize=True) + np.einsum('mnei,abjn->abmije',Voovo,t2,optimize=True) -\
                np.einsum('mnei,abnj->abmije',np.einsum('mnef,fi->mnei',Voovv,t1,optimize=True),t2,optimize=True) + \
                np.einsum('mnej,abni->abmije',np.einsum('mnef,fj->mnej',Voovv,t1,optimize=True),t2,optimize=True) + \
                np.einsum('mbef,afij->abmije',Vovvv,t2,optimize=True) - np.einsum('maef,bfij->abmije',Vovvv,t2,optimize=True) - \
                np.einsum('maef,fbij->abmije',np.einsum('mnef,an->maef',Voovv,t1,optimize=True),t2,optimize=True) + \
                np.einsum('mbef,faij->abmije',np.einsum('mnef,bn->mbef',Voovv,t1,optimize=True),t2,optimize=True)
        
    W_amnije = np.einsum('mnfe,afij->amnije',Voovv,t2,optimize=True)
    
    W_abmief = -np.einsum('mnfe,abin->abmief',Voovv,t2,optimize=True)
    
    # Calculate matrix-vector products of Hbar*R
    
    # <ia|HR_1|0>
    TEMP1 = np.einsum('ae,ei->ai',F_ae,r1,optimize=True) - np.einsum('mi,am->ai',F_mi,r1,optimize=True) + \
            np.einsum('maei,em->ai',W_mbej,r1,optimize=True)
        
    # <ia|HR_2|0>
    TEMP2 = 0.5*np.einsum('amfe,feim->ai',W_amef,r2,optimize=True) - 0.5*np.einsum('mnie,aemn->ai',W_mnie,r2,optimize=True) + \
            np.einsum('me,aeim->ai',F_me,r2,optimize=True)
        
    # <ijab|HR_1|0>
    TEMP3 = np.einsum('abej,ei->abij',W_abei,r1,optimize=True) - np.einsum('abei,ej->abij',W_abei,r1,optimize=True) - \
            np.einsum('mbij,am->abij',W_mbij,r1,optimize=True) + np.einsum('maij,bm->abij',W_mbij,r1,optimize=True) + \
            np.einsum('abmije,em->abij',W_abmije,r1,optimize=True)
            
    # <ijab|HR_2|0>  
    TEMP4 = np.einsum('bd,adij->abij',F_ae,r2,optimize=True) - np.einsum('ad,bdij->abij',F_ae,r2,optimize=True) - \
            np.einsum('mj,abim->abij',F_mi,r2,optimize=True) + np.einsum('mi,abjm->abij',F_mi,r2,optimize=True) + \
            0.5*np.einsum('abef,efij->abij',W_abef,r2,optimize=True) + 0.5*np.einsum('mnij,abmn->abij',W_mnij,r2,optimize=True) + \
            np.einsum('mbej,aeim->abij',W_mbej,r2,optimize=True) - np.einsum('mbei,aejm->abij',W_mbej,r2,optimize=True) - np.einsum('maej,beim->abij',W_mbej,r2,optimize=True) + np.einsum('maei,bejm->abij',W_mbej,r2,optimize=True) - \
            0.5*np.einsum('amnije,bemn->abij',W_amnije,r2,optimize=True) + 0.5*np.einsum('bmnije,aemn->abij',W_amnije,r2,optimize=True) + \
            0.5*np.einsum('abmief,efjm->abij',W_abmief,r2,optimize=True) - 0.5*np.einsum('abmjef,efim->abij',W_abmief,r2,optimize=True)

    # Construct sigma trial vectors for Davidson iterations
    sigma_r1 = TEMP1 + TEMP2
    sigma_r2 = TEMP3 + TEMP4
    
    return sigma_r1, sigma_r2

def calculate_r0(r1,r2,t1,t2,VM,FM,Nocc,Nunocc,omega):
    
    fov = FM[:Nocc,Nocc:]
    Voovv = VM[:Nocc,:Nocc,Nocc:,Nocc:]
    
    F_me = fov + np.einsum('mnef,fn->me',Voovv,t1,optimize=True)
    
    r0 = 0
    r0 += np.einsum('ia,ai->',F_me,r1,optimize=True)
    r0 += 0.25*np.einsum('ijab,abij->',Voovv,r2,optimize=True)
    r0 *= 1/omega
    
    return r0


In [12]:
def gramschmidt_v3(A):
    k = A.shape[1]
    A[:,0] = A[:,0]/np.linalg.norm(A[:,0])
    for i in range(1,k):
        for j in range(i):
            A[:,i] -= np.dot(A[:,i],A[:,j])*A[:,j]
        A[:,i] = A[:,i]/np.linalg.norm(A[:,i])
    return A

def gramschmidt_v2(A,tol):
    k = A.shape[1]
    A[:,0] = A[:,0]/np.linalg.norm(A[:,0])
    B = A[:,0][:,np.newaxis]
    ct = 0
    for i in range(1,k):
        ct += 1
        for j in range(ct):
            A[:,ct] -= np.dot(A[:,ct],A[:,j])*A[:,j]
        normA = np.linalg.norm(A[:,ct])
        if normA > tol:
            A[:,ct] = A[:,ct]/normA
            B = np.concatenate((B,A[:,ct][:,np.newaxis]),axis=1)
        else:
            np.delete(A,ct,axis=1)
            ct -= 1
    return B

def gramschmidt_v3(A,q):
    n = A.shape[0]
    TMP = np.zeros((n,n))
    I = np.eye(n)
    for i in range(A.shape[1]):
        TMP *= I - np.dot(A[:,i],A[:,i].T)
    q2 = TMP*q
    return q2/np.linalg.norm(q2)
    

# def eomcc_init_guess(D,nstates,nvec_per_state,VM,FM):
    
#         if not flag_cis_guess:
#             B_idx = D.argsort()[:nstates*nvec_per_state]
#             B = np.eye(Hbar_dim)[:,B_idx]
#             omega = D[B_idx]
#         else:
#             H_CIS = cis_hamiltonian(FM,VM,H00,occ,unocc)
#             omega,B = cis_guess(H_CIS,Nocc,Nunocc)
#             omega = omega[:nstates*nvec_per_state]
#             B = np.vstack((B[:,:nstates*nvec_per_state],np.zeros((Nov*Nov,nstates*nvec_per_state)))) 

def eomcc_block_davidson(B, D, t1, t2, VM, FM, nstates, \
                         nvec_per_state, max_nvec_per_state, maxit_eom, tol_eom, thresh_vec):

    
        eom_conv = np.zeros(nstates)
        
        Nunocc = t1.shape[0]
        Nocc = t1.shape[1] 
        Nov = Nocc*Nunocc
        Hbar_dim = Nov + Nov*Nov
        
        L = nstates*nvec_per_state
        Lmax = nstates*max_nvec_per_state
        
        omega = [0.0]*L

#         B_idx = D[:Nov].argsort()[:L]
#         B = np.eye(Hbar_dim)[:, B_idx]

        it_eom = 1; flag_conv_eom = False; 
        while it_eom < maxit_eom:
            
            # Orthonormalize search space
            B = gramschmidt_v2(B,thresh_vec)
            #B, _ = np.linalg.qr(B)
            
            # Update search space dimension
            L = B.shape[1]
            if it_eom > 1:
                if L_old == L:
                    break
            
            # Store old eigvalues 
            omega_old = omega.copy()
            
            print('Iter-{}    L = {}'.format(it_eom,L))
            print('----------------------------------')
            
            # Matrix-vector product
            SIGMA = np.zeros_like(B)
            for i in range(L):
                B1 = B[:Nov,i].reshape(Nunocc,Nocc)
                B2 = B[Nov:,i].reshape(Nunocc,Nunocc,Nocc,Nocc)
                sigma_r1,sigma_r2 = sigma_fcn(B1,B2,t1,t2,VM,FM,Nocc,Nunocc)
                SIGMA[:Nov,i] = sigma_r1.flatten()
                SIGMA[Nov:,i] = sigma_r2.flatten()

            # Interaction matrix
            G = np.dot(B.T,SIGMA)
            
            # Sort eigpairs
            omega, alpha = np.linalg.eig(G)
            idx = omega.argsort()[:nstates]
            omega = omega[idx]
            alpha = alpha[:, idx]
            
            Q = np.zeros((Hbar_dim, nstates))
            resid_norm = np.zeros(nstates)
            for j in range(nstates):

                # Calculate residual
                r = np.dot(SIGMA, alpha[:, j]) - omega[j]*np.dot(B, alpha[:,j])
                resid_norm[j] = np.linalg.norm(r)

                # Precondition residual
                Q[:,j] = r/(omega[j]-D[j])
               
                # Print status
                print('    Root = {}     e = {:>10.12f}    |r| = {:>10.12f}'.\
                      format(j+1,np.real(omega[j]),resid_norm[j]))
        
            
            # Check convergence
            res = np.sum(resid_norm)/nstates
            eps = np.sqrt(np.sum( (omega[:nstates]-omega_old[:nstates])**2 ))
            if res < tol_eom and eps < 1.0e-6:
                flag_conv_eom = True
                break
            else:
                if L >= Lmax:
                    print('Restarting and Collapsing...')
                    #B = Rz[:,:nstates].copy()
                    B = np.dot(B,alpha[:,:nstates])
                    omega = omega_old.copy()
                else:
                    B = np.concatenate((B,Q),axis=1)
                    L_old = L
                
                    
            it_eom +=1 
            
        for j in range(nstates):
            if resid_norm[j] < tol_eom:
                eom_conv[j] = 1
        
        if flag_conv_eom:
            
            print('Completed EOM-CCSD Routine in {} iterations ({} seconds)'.format(it_eom, time.time()-t0))
            for j in range(nstates):
                print('E'+str(j+1)+'_exc = {} Ha\n'.format(np.real(omega[j])))
        else:
            print('EOM-CCSD not converged in {} iterations\n'.format(maxit_eom))
            
        return omega, SIGMA, eom_conv

In [13]:
def eomcc_block_davidson_v2(V0, D, Rz_prev, omega0, t1, t2, VM, FM, n, max_nvec_per_state, maxit_eom, tol_eom, thresh_vec, root_homing):

        Nunocc = t1.shape[0]
        Nocc = t1.shape[1] 
        Nov = Nocc*Nunocc
        Hbar_dim = Nov + Nov*Nov
        eom_conv = 0
        
        
        B = V0.copy()
        #omega = omega0.copy()
        omega = [0.0] * len(omega0)
            
        print('Root-{} :'.format(n+1))
        print('---------------------------------------------------------------')
        print('Iter - {}  Initial Guess = {:>10.12f}'.format(0,omega0))


        it_eom = 1; flag_conv_eom = False;
        while it_eom < maxit_eom:

            # Orthonormalize search space
            B,_ = np.linalg.qr(B)
            #if n > 0:
                #B = gramschmidt_v2(np.concatenate((B,Rz_prev),axis=1),thresh_vec)
                
                #B,_ = np.linalg.qr(np.concatenate((B,Rz_prev),axis=1))
                #B = B[:,:B.shape[1]-Rz_prev.shape[1]]
                
            #    B,_ = np.linalg.qr(B)
            #else:
                #B = gramschmidt_v2(B,thresh_vec)

            # Update search space dimension
            L = B.shape[1]
            #if it_eom > 1:
            #    if L_old == L:
            #        break

            # Store old eigvalues 
            omega_old = omega.copy()

            # Matrix-vector product
            SIGMA = np.zeros_like(B)
            for i in range(L):
                B1 = B[:Nov,i].reshape(Nunocc,Nocc)
                B2 = B[Nov:,i].reshape(Nunocc,Nunocc,Nocc,Nocc)
                sigma_r1,sigma_r2 = sigma_fcn(B1,B2,t1,t2,VM,FM,Nocc,Nunocc)
                SIGMA[:Nov,i] = sigma_r1.flatten()
                SIGMA[Nov:,i] = sigma_r2.flatten()

            # Interaction matrix
            G = np.dot(B.T,SIGMA)

            # Sort eigpairs
            # The problem is here - our Ritz vectors are not at all like the CIS
            # or true eigenvectors of HBAR
            omega, alpha = np.linalg.eig(G)
            alpha = np.real(alpha)
            #alpha = gramschmidt_v2(alpha,0.0)
            omega = np.real(omega)

            #X = np.dot(B,alpha)
            trial_overlap = np.abs(alpha[0,:])
            trial_energy_distance = np.abs(omega - omega0*np.ones(len(omega)))
        
            if root_homing == 'energy':
                idx = np.argsort(trial_energy_distance)
                P = idx[0]
            if root_homing == 'overlap':
                idx = np.argsort(trial_overlap)
                P = idx[-1]
            
            omega = omega[P]
            alpha = alpha[:,P]

            # Ritz vector
            Rz = np.dot(B,alpha)

            # Calculate residual
            r = np.dot(SIGMA, alpha) - omega*Rz
            resid_norm = np.linalg.norm(r)

            # Precondition residual
            Q = r/(omega-D[n])

            # Print status
            print('Iter - {}    L = {}     e = {:>10.12f}    |r| = {:>10.12f}'.\
                  format(it_eom, L, np.real(omega), resid_norm))


            # Check convergence
            if resid_norm < tol_eom:
                flag_conv_eom = True
                eom_conv = 1
                break
            else:
                if L >= max_nvec_per_state:
                    print('Restarting and Collapsing...')
                    B = np.dot(B,alpha)[:,np.newaxis]
                    omega = omega_old.copy()
                    #L_old = L
                else:
                    #Q = gramschmidt_v3(B,Q)
                    if len(Q.shape) > 1:
                        B = np.concatenate((B,Q),axis=1)
                    else:
                        B = np.concatenate((B,Q[:,np.newaxis]),axis=1)
                #L_old = L


            it_eom +=1 
          
        if flag_conv_eom:
            print('Root {} converged in {} iterations'.format(n+1,it_eom))
            print('\n')
        else:
            print('Root {} not converged in {} iterations'.format(n+1,it_eom))
            print('\n')
            
        return omega, Rz, eom_conv, B

In [14]:
def bse_parse(basis,Z):
    import basis_set_exchange as bse
    BS = bse.get_basis(basis,elements=[Z],fmt='gamess_us',header=False)
    shell = []
    xi = []
    Cm = []
    BSS = BS.splitlines()[3:-1]
    for line in BSS:
        x = line.split()
        if x[0] == 'S' or x[0] == 's':
            SH = 'S'
            NGS = int(x[1])
            shell.append((0,0,0))
            xi_temp = []
            Cm_temp = []
        elif x[0] == 'P' or x[0] == 'p':
            SH = 'P'
            NGS = int(x[1])
            shell.append((1,0,0))
            shell.append((0,1,0))
            shell.append((0,0,1))
            xi_temp = []
            Cm_temp = []
        elif x[0] == 'D' or x[0] == 'd':
            SH = 'D'
            NGS = int(x[1])
            shell.append((2,0,0))
            shell.append((0,2,0))
            shell.append((0,0,2))
            shell.append((1,1,0))
            shell.append((0,1,1))
            shell.append((1,0,1))
            xi_temp = []
            Cm_temp = []
        elif x[0] == 'L' or x[0] == 'l':
            SH = 'L'
            NGS = int(x[1])
            shell.append((0,0,0))
            shell.append((1,0,0))
            shell.append((0,1,0))
            shell.append((0,0,1))
            xi_temp = []
            Cm_temp_s = []
            Cm_temp_p = []
        else:
            if SH != 'L':
                xi_temp.append(float(x[1]))
                Cm_temp.append(float(x[2]))
            else:
                xi_temp.append(float(x[1]))
                Cm_temp_s.append(float(x[2]))
                Cm_temp_p.append(float(x[3]))
            if len(xi_temp) == NGS:
                if SH != 'L':
                    if SH == 'S':
                        xi.append(xi_temp)
                        Cm.append(Cm_temp)
                    elif SH == 'P':
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                    elif SH == 'D':
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                    else:
                        print('Unknown shell label')
                else:
                    xi.append(xi_temp)
                    xi.append(xi_temp)
                    xi.append(xi_temp)
                    xi.append(xi_temp)
                    Cm.append(Cm_temp_s)
                    Cm.append(Cm_temp_p)
                    Cm.append(Cm_temp_p)
                    Cm.append(Cm_temp_p)
    return shell, xi, Cm

def get_basis(basis,Z):
    SHELL = []
    XI = []
    CM = []
    for z in Z:
        shell,xi,Cm = bse_parse(basis,z)
        SHELL.append(shell)
        XI.append(xi)
        CM.append(Cm)
    return SHELL,XI,CM

In [15]:
# Get the basis set for the calculation
SHELL, XI, CM = get_basis(basis,Z)
Norb = len(list(itertools.chain.from_iterable(XI)))

In [16]:
# Initialize containers
EHF = np.zeros(num_pts); ECC = np.zeros(num_pts); 
EEOM = np.zeros((nstates,num_pts)); eom_conv = np.zeros((nstates,num_pts))
Coeff_list = []; Pmat_list = []; MOeps_list = []; orbs_list = []; 

# Number of occupied spatial orbitals (RHF, closed shell case)
if Nelec > 1:
    Nocc0 = int(Nelec/2)
else:
    Nocc0 = int(Nelec)
    

print('System - {}'.format(molecule))
print('-----------------------------')
print('Number of electrons - {}'.format(Nelec))
print('Basis Set - {}'.format(basis))
print('Number of atomic orbitals - {}'.format(Norb))
print('\n')


# Coordinate sweep
for J in range(len(RJ)):
    
    ###### Printing System Information #######
    
    Rp = RJ[J]
    print('Atomic Coordiantes (a.u.) - R = {}'.format(Rp))
    for at in range(len(atom_labels)):
        print('{}:     {:4.6}     {:4.6}     {:4.6}'.format(atom_labels[at], \
                                                float(atom_coordinates[J][at][0]), float(atom_coordinates[J][at][1]), float(atom_coordinates[J][at][2])))
    print('\n')
    
    
    ####### Integral Calculation #########

    print('Beginning AO integral evaluation...')
    t0 = time.time()
    
    # Construct atom-centered CGO basis functions
    orbs = construct_orbitals(SHELL, XI, CM, atom_coordinates[J])
    Smat,Tmat,Vmat = calculate_onebody_ints(orbs,atom_coordinates[J])
    VVmat = calculate_twobody_ints(orbs,0)
 
    print('Completed AO integral construction ({:>.4} seconds)\n'.format(time.time()-t0))
                    
    ####### SCF Routine #########
    
    print('Beginning SCF routine...')
    t0 = time.time()
    
    # Form Hcore
    Hcore = Tmat + Vmat
    
    # Orthogonalization of basis
    X = orthomat(Smat,0.0,'symmetric')
    
    # Initial guess at P 
    P = np.zeros((Norb,Norb))
    
    # Calculate Fock matrix with guess
    G = np.einsum('prqs,rs->pq',VVmat,P,optimize=True)-\
        0.5*np.einsum('prsq,rs->pq',VVmat,P,optimize=True)
    Fock = Hcore + G + level_shift*(Smat-Smat*P*Smat)
    
    eps_mo, Cp = np.linalg.eigh(np.dot(X.T,np.dot(Fock,X)))
    idx = eps_mo.argsort(); eps_mo = eps_mo[idx]
    C = dot(X,Cp[:,idx])
    P = 2*np.einsum('pi,qi->pq',C[:,:Nocc0],C[:,:Nocc0],optimize=True)
    
    # DIIS containers
    F_list = []; resid_F = []
    
    # SCF Iterations
    it = 1; flag_conv_scf = False
    while it < maxit_scf: 
        
        # Calculate Fock matrix with guess
        G = np.einsum('prqs,rs->pq',VVmat,P,optimize=True)-\
            0.5*np.einsum('prsq,rs->pq',VVmat,P,optimize=True)
        Fock = Hcore + G + level_shift*(Smat-Smat*P*Smat)
        
        # Build DIIS Residual
        diis_r = X.dot(Fock.dot(P).dot(Smat) - Smat.dot(P).dot(Fock)).dot(X)
        scf_resid = np.mean(diis_r**2)**0.5
        
        if scf_resid <= tol_scf:
            flag_conv_scf = True
            it_scf = it
            break
            
        # Append trial & residual vectors to lists
        # (Note: do NOT use extrapolated matrices in F_list!)
        if len(F_list) == diis_size_scf:
            del F_list[0]
            del resid_F[0]
        F_list.append(Fock)
        resid_F.append(diis_r)
        
        # DIIS extrapolated Fock matrix
        if it >= 2:
            Fock = diis_pulay_solver(F_list, resid_F)
    
        # Compute new orbital guess with DIIS Fock matrix
        eps_mo, Cp = np.linalg.eigh(np.dot(X.T,np.dot(Fock,X)))
        idx = eps_mo.argsort(); eps_mo = eps_mo[idx]
        C = dot(X,Cp[:,idx])
        
        # Compute HF energy and density matrix 
        P = 2*np.einsum('pi,qi->pq',C[:,:Nocc0],C[:,:Nocc0],optimize=True)
        H00 = 0.5*np.einsum('qp,pq->',P,Hcore+Fock,optimize=True) + calc_nuclear_nuclear(atom_coordinates[J],Z)
        
        if flag_scf_verbose:
            print('Iter - {}    Residuum = {:>10.12f}    Escf = {:>10.12f}'.format(it, scf_resid, H00))
                
        it += 1
        
    # Compute new orbital guess with DIIS Fock matrix
    eps_mo, Cp = np.linalg.eigh(np.dot(X.T,np.dot(Fock,X)))
    idx = eps_mo.argsort(); eps_mo = eps_mo[idx]
    C = dot(X,Cp[:,idx])

    # Compute HF energy and density matrix 
    P = 2*np.einsum('pi,qi->pq',C[:,:Nocc0],C[:,:Nocc0],optimize=True)
            
    # Store charge density, coefficients, and HF energy for given geometry
    Pmat_list.append(P)
    Coeff_list.append(C)
    MOeps_list.append(eps_mo)
    orbs_list.append(orbs) 
    EHF[J] = H00
    
    if flag_conv_scf:
        print('Completed SCF Routine in {} iterations ({:>.4} seconds)'.format(it_scf, time.time()-t0))
        print('Erhf = {} Ha\n'.format(EHF[J]))

        ####### Post-HF Processing #########
        t0 = time.time()
        
        # Convert AO integrals to MO integrals
        VVmat_mo = ao_to_mo(VVmat,C); 
        Fock_mo = ao_to_mo(Fock,C)
        Hcore_mo = ao_to_mo(Hcore,C)

        # Convert MO spatial integrals to spinorbital integrals
        VM = spatial_to_spinorb(VVmat_mo)
        FM = spatial_to_spinorb(Fock_mo)
        ZM = spatial_to_spinorb(Hcore_mo)

        # Antisymmetrize two-body integral matrix
        VM = VM - np.einsum('pqrs->pqsr',VM) 

        # Occupied and unoccuped indices
        occ = [occ for occ in range(2*Nocc0)]; 
        unocc = [unocc for unocc in range(2*Nocc0,2*Norb)]; 
        Nocc = len(occ)
        Nunocc = len(unocc)
        
        print('AO to MO conversion took {:>.4} seconds\n'.format(time.time()-t0))
        
        ####### CCSD Routine #########
        print('Beginning CCSD routine...')

        t0 = time.time()

        # Initialize T1 and T2 and DIIS containers
        t1 = np.zeros((Nunocc,Nocc))
        t2 = np.zeros((Nunocc,Nunocc,Nocc,Nocc))
        t1_list = []; t2_list = [];
        t1_resid_list = []; t2_resid_list = [];

        # Jacobi/DIIS iterations
        it = 1; flag_conv_cc = False
        while it < maxit_cc: # T1/T2 iterations

            # Store old t1 and t2
            t1_old = t1.copy()
            t2_old = t2.copy()

            # Update t1 and t2 by Jacobi
            t1 = t1_update(t1,t2,VM,FM,occ,unocc)
            t2 = t2_update(t1,t2,VM,FM,occ,unocc)

            # Build DIIS Residual
            resid_t1 = t1 - t1_old
            resid_t2 = np.reshape(t2 - t2_old,(Nunocc**2,Nocc**2))
            ccsd_resid = np.mean(resid_t1**2)**0.5 + np.mean(resid_t2**2)**0.5
            # Check for exit condition
            if ccsd_resid < tol_cc:
                flag_conv_cc = True
                it_cc = it
                break

            # Append trial & residual vectors to lists
            # (Note: Do NOT use extrapolated t1/t2 to build these!)
            if len(t1_list) == diis_size_cc:
                del t1_list[0]
                del t1_resid_list[0]
            if len(t2_list) == diis_size_cc:
                del t2_list[0]
                del t2_resid_list[0]
            t1_list.append(t1)
            t1_resid_list.append(resid_t1)
            t2_list.append(t2)
            t2_resid_list.append(resid_t2)

            # DIIS extrapolated T1 and T2
            if it >= 2:
                t1 = diis_pulay_solver(t1_list,t1_resid_list)
                t2 = diis_pulay_solver(t2_list,t2_resid_list)
                
            # CC correlation energy
            Ecorr = cc_energy(t1,t2,VM,FM,Nocc)
            
            if flag_cc_verbose:
                print('Iter - {}    Residuum = {:>10.12f}    Ecorr = {:>10.12f}'.format(it, ccsd_resid, Ecorr))

            it += 1

        # Total energy
        ECC[J] = EHF[J] + Ecorr

        if flag_conv_cc:
            print('Completed CCSD Routine in {} iterations ({:>.4} seconds)'.format(it_cc, time.time()-t0))
            print('Eccsd = {} Ha'.format(ECC[J]))
            print('Ecorr = {} Ha\n'.format(Ecorr))
        else:
            print('CCSD not converged in {} iterations\n\n'.format(maxit_cc))
            
        ####### Left-CCSD Routine ########
        print('Beginning Left-CCSD routine...')

        t0 = time.time()

        # Initialize lambda1 and lambda2 and DIIS containers
        lambda1 = np.zeros((Nocc,Nunocc))
        lambda2 = np.zeros((Nocc,Nocc,Nunocc,Nunocc))
        lambda1_list = []; lambda2_list = [];
        lambda1_resid_list = []; lambda2_resid_list = [];

        # Jacobi/DIIS iterations
        it = 1; flag_conv_lcc = False
        while it < maxit_cc: # lambda1/lambda2 iterations

            # Store old t1 and t2
            lambda1_old = lambda1.copy()
            lambda2_old = lambda2.copy()

            # Update lambda1 and lambda2 by Jacobi
            lambda1 = lambda1_update(lambda1,lambda2,t1,t2,VM,FM,occ,unocc)
            lambda2 = lambda2_update(lambda1,lambda2,t1,t2,VM,FM,occ,unocc)

            # Build DIIS Residual
            resid_lambda1 = lambda1 - lambda1_old
            resid_lambda2 = np.reshape(lambda2 - lambda2_old,(Nocc**2,Nunocc**2))
            lccsd_resid = np.mean(resid_lambda1**2)**0.5 + np.mean(resid_lambda2**2)**0.5
            # Check for exit condition
            if lccsd_resid < tol_cc:
                flag_conv_lcc = True
                it_cc = it
                break

            # Append trial & residual vectors to lists
            # (Note: Do NOT use extrapolated t1/t2 to build these!)
            if len(lambda1_list) == diis_size_cc:
                del lambda1_list[0]
                del lambda1_resid_list[0]
            if len(lambda2_list) == diis_size_cc:
                del lambda2_list[0]
                del lambda2_resid_list[0]
            lambda1_list.append(lambda1)
            lambda1_resid_list.append(resid_lambda1)
            lambda2_list.append(lambda2)
            lambda2_resid_list.append(resid_lambda2)

            # DIIS extrapolated T1 and T2
            if it >= 2:
                lambda1 = diis_pulay_solver(lambda1_list,lambda1_resid_list)
                lambda2 = diis_pulay_solver(lambda2_list,lambda2_resid_list)
            
            if flag_cc_verbose:
                print('Iter - {}    Residuum = {:>10.12f}'.format(it, lccsd_resid))

            it += 1

        if flag_conv_lcc:
            print('Completed Left-CCSD Routine in {} iterations ({:>.4} seconds)'.format(it_cc, time.time()-t0))
#             print('Eccsd = {} Ha'.format(ECC[J]))
#             print('Ecorr = {} Ha\n'.format(Ecorr))
            print('\n')
        else:
            print('Left-CCSD not converged in {} iterations\n\n'.format(maxit_cc))
            print('\n')
            
        ####### EOM-CCSD Routine #########
        
        print('Beginning EOM-CCSD Routine...')
        
        Nov = Nocc*Nunocc
        Noovv = Nov**2
        Hbar_dim = Nov + Noovv
            
        t0 = time.time()

        D = Hbar_diagonal(t1,t2,VM,FM,occ,unocc)
        
#         E_exc, Rz = davidson_v3(D, t1, t2, VM, FM, Nocc, Nunocc, nstates, nvec_per_state, maxit_eom, tol_eom)
        
        if not flag_cis_guess:
            B_idx = D.argsort()[:nstates*nvec_per_state]
            B = np.eye(Hbar_dim)[:,B_idx]
            omega = D[B_idx]
        else:
            H_CIS = cis_hamiltonian(FM,VM,H00,occ,unocc)
            omega_cis, B = cis_guess(H_CIS,Nocc,Nunocc)
            omega_cis = omega_cis[:nstates*nvec_per_state]
            B = np.vstack((B[:,:min(nstates*nvec_per_state,B.shape[0])], \
                                     np.zeros((Noovv,min(nstates*nvec_per_state,B.shape[0]))))) 
            print('CIS Excitation Energies - ')
            for i in range(len(omega_cis)):
                print('Root {}: {:>5.7}'.format(i+1, omega_cis[i]))
            print('\n')
        
        if eigensolver == 1:
            E_exc, Rz, eom_root_conv = eomcc_block_davidson(B, D, t1, t2, VM, FM, nstates, \
                                  nvec_per_state, max_nvec_per_state, maxit_eom, tol_eom, thresh_vec)
        else:
            E_exc = np.zeros(nstates)
            eom_root_conv = np.zeros(nstates)
            Rz = np.zeros((Hbar_dim, nstates))
            for state in range(nstates):
                if state == 0:
                    V0 = B[:,state*nvec_per_state:(state+1)*nvec_per_state]
                theta, rz, conv, V0 = eomcc_block_davidson_v2(V0, D, Rz[:,:state], omega[state], t1, t2, VM, FM, state, \
                                                max_nvec_per_state, maxit_eom, tol_eom, thresh_vec, root_homing)
                E_exc[state] = np.real(theta)
                eom_root_conv[state] = conv
                Rz[:,state] = np.squeeze(rz)

        EEOM[:,J] =  np.squeeze(np.tile(ECC[J],(nstates,1))) + np.real(E_exc)
        eom_conv[:,J] = eom_root_conv
        
        
    else:
        print('SCF not converged in {} iterations\n\n'.format(maxit_scf))



System - H2O
-----------------------------
Number of electrons - 10
Basis Set - 6-31g
Number of atomic orbitals - 13


Atomic Coordiantes (a.u.) - R = 0.5
H:     0.395345     0.306109      0.0
H:     -0.395345     0.306109      0.0
O:      0.0      0.0      0.0


Beginning AO integral evaluation...
Completed AO integral construction (84.23 seconds)

Beginning SCF routine...
Completed SCF Routine in 13 iterations (0.03905 seconds)
Erhf = -65.88394732169402 Ha

AO to MO conversion took 1.155 seconds

Beginning CCSD routine...
Iter - 1    Residuum = 0.001450395145    Ecorr = -0.061409986872
Iter - 2    Residuum = 0.001913982141    Ecorr = -0.066026702350
Iter - 3    Residuum = 0.000346487727    Ecorr = -0.067101690005
Iter - 4    Residuum = 0.000351844012    Ecorr = -0.066988440824
Iter - 5    Residuum = 0.000343444424    Ecorr = -0.066983721012
Iter - 6    Residuum = 0.000342674535    Ecorr = -0.066985360664
Iter - 7    Residuum = 0.000309820737    Ecorr = -0.066986102162
Iter - 8    Res

KeyboardInterrupt: 

In [ ]:
def plotfunction_diatomic(x,y,basis,labels):
    #bohrtoang = 0.529177
    bohrtoang = 1
    plt.rcParams.update({'font.size': 20})
    fig = plt.figure(figsize = (12,8))
    for i in range(len(y)):
        plt.plot(x[i][:J]*bohrtoang,y[i][:J],'-.')
        plt.scatter(x[i][:J]*bohrtoang,y[i][:J],20,label=labels[i])
    plt.grid(True,alpha=0.25)
    plt.xlabel(r'$r$  [bohr]')
    plt.ylabel(r'$E$  $[Ha]$')
    plt.legend(loc=1,fontsize=20)
    #plt.ylim([-76.2,-75.95])
    #plt.xlim([0.7,1.45])
    plt.figtext(0.78, 0.7, 'Basis set: '+basis, wrap=True, horizontalalignment='center', fontsize=19)
    
    iconv = np.logical_not(np.isnan(y[-1]))
    Econv = y[-1][iconv]
    RJconv = RJ[iconv]
    Ieq = np.argmin(Econv)
    Req = RJconv[Ieq]
    Eeq = Econv[Ieq]

    print('The bond length is {} bohr ({} angstrom)'.format(Req,Req*bohrtoang))
    print('The bond energy is {} Hartree'.format(Eeq))
    
def plotfunction_diatomic_exc(x,y,basis,eom_conv,labels):
    bohrtoang = 0.529177
    bohrtoang2 = 1
    p = []
    ct = 0
    leg = []
    
    for ll in labels[0]:
        leg.append(ll)
    leg.append(labels[1])
    
    plt.rcParams.update({'font.size': 20})
    fig = plt.figure(figsize = (12,8))
    for i in range(len(y)):
        if y[i].ndim != 1:
            for j in range(y[i].shape[0]):
                p.append(plt.plot(x[i][:J]*bohrtoang2,y[i][j,:J],'-'))
                plt.scatter(x[i][np.argwhere(eom_conv[j,:J]==1)]*bohrtoang2,y[i][j,np.argwhere(eom_conv[j,:J]==1)],10)
                #plt.scatter(x[i][np.argwhere(eom_conv[j,:J]==0)]*bohrtoang2,y[i][j,np.argwhere(eom_conv[j,:J]==0)],10,facecolors='none')
                if not len(labels[i][j]) == 0:
                    print(labels[i][j])
                    Ieq = np.argmin(y[i][j,:J])
                    Req = x[i][Ieq]
                    Eeq = y[i][j,Ieq]
                    print('The bond length is {} bohr ({} angstrom)'.format(Req,Req*bohrtoang))
                    print('The bond energy is {} Hartree\n'.format(Eeq))
        else:
            p.append(plt.plot(x[i][:J]*bohrtoang2,y[i][:J],'-'))
            plt.scatter(x[i][:J]*bohrtoang2,y[i][:J],10)
            if not len(labels[i]) == 0:
                print(labels[i])
                Ieq = np.argmin(y[i])
                Req = x[i][Ieq]
                Eeq = y[i][Ieq]
                print('The bond length is {} bohr ({} angstrom)'.format(Req,Req*bohrtoang))
                print('The bond energy is {} Hartree\n'.format(Eeq))
            
    plt.grid(True,alpha=0.25)
    plt.xlabel(r'$r$  [bohr]')
    plt.ylabel(r'$E$  $[Ha]$')
    plt.legend(leg,fontsize=20,bbox_to_anchor=[1.165, 0.54],loc='center')
    #plt.ylim([-1.25,0.5])
    #plt.xlim([2.38,2.42])
    plt.figtext(0.78, 0.15, 'Basis set: '+basis, wrap=True, horizontalalignment='center', fontsize=19)
    
def plotfunction_angular(x,y,basis,labels):   
    plt.rcParams.update({'font.size': 20})
    fig = plt.figure(figsize = (12,8))
    for i in range(len(y)):
        plt.plot(np.degrees(x[i]),y[i],'-.')
        plt.scatter(np.degrees(x[i]),y[i],20,label=labels[i])
    plt.grid(True,alpha=0.25)
    plt.xlabel(r'$\theta$ [degrees]')
    plt.ylabel(r'$E$  $[Ha]$')
    plt.legend(loc=4,fontsize=20)
    plt.figtext(0.80, 0.27, 'Basis set: '+basis, wrap=True, horizontalalignment='center', fontsize=19)
    
    iconv = np.logical_not(np.isnan(y[-1]))
    Econv = y[-1][iconv]
    RJconv = RJ[iconv]
    Ieq = np.argmin(Econv)
    Req = np.degrees(RJconv[Ieq])
    Eeq = Econv[Ieq]

    print('The bond angle is {} degrees'.format(Req))
    print('The bond energy is {} Hartree'.format(Eeq))

In [ ]:
lab_ex = ['E'+str(i+1) for i in range(nstates)]
labs = [lab_ex,'E0']
plotfunction_diatomic_exc([RJ,RJ],[EEOM,ECC],basis,eom_conv,labs)

In [ ]:
def MO_wavefunctions_diatomic(nn):

#   nn = 500

    xdom = np.array([0])
    ydom = np.linspace(-5,5,nn)
    zdom = np.linspace(-5,5,nn)

    psiMO = [[[] for i in range(Norb)] for j in range(len(RJ))]
    rhoMO = [[[] for i in range(Norb)] for j in range(len(RJ))]

    for J in range(len(RJ)):
        Gt = np.zeros((Norb,len(xdom)*len(ydom)*len(zdom)))
        for k in range(Norb):
            G = orbs_list[J][k].basisfcn_matrix(xdom,ydom,zdom)
            Gt[k,:] = np.reshape(G,(len(xdom)*len(ydom)*len(zdom)))
        temp = Coeff_list[J].T@Gt
        for j in range(Norb):
            psiMO[J][j] = np.reshape(temp[j,:],(len(xdom),len(ydom),len(zdom)))
            rhoMO[J][j] = np.conj(psiMO[J][j])*psiMO[J][j]
            
    return psiMO, rhoMO

def plot_MO_diatomic(psiMO, rhoMO, num_orb, m, nn, Rspec):
    
    import math
    def truncate(number, digits) -> float:
        stepper = 10.0 ** digits
        return math.trunc(stepper * number) / stepper
    plt.rcParams['font.family'] = 'fantasy'
    plt.rcParams['font.fantasy'] = ['Comic Sans MS']
    plt.rcParams.update({'font.size': 20})
    fig = plt.figure(figsize = (12,8))
    
    if len(Rspec) == 0:
        for i in range(0,len(RJ),m):
            #print('R = {}'.format(RJ[i]))
            plt.plot(zdom,np.squeeze(rhoMO[i][num_orb-1])[:,int(nn/2)],label='r = '+str(truncate(RJ[i],3))+' a.u.')
    else:
        for i in range(len(Rspec)):
            I = np.argmin(np.abs(RJ-Rspec[i]))
            plt.plot(zdom,np.squeeze(rhoMO[I][num_orb-1])[:,int(nn/2)],label='r = '+str(truncate(RJ[I],3))+' a.u.')
    plt.legend(loc=1,fontsize=15)
    #plt.grid(True)
    plt.xlabel('r / a.u.')
    plt.ylabel(r'$|\psi_{MO}(r)|^2$')
    plt.figtext(0.23, 0.84, 'Molecule: '+molecule, wrap=True, horizontalalignment='center', fontsize=19)
        #plt.contourf(zdom,ydom,np.squeeze(psiMO[i][0]).T)

In [ ]:
nn = 500
psiMO, rhoMO = MO_wavefunctions_diatomic(nn)

In [ ]:
plot_MO_diatomic(psiMO,rhoMO, 2, 10, nn, [])


In [ ]:
# function [x, error, iter, flag] = gmres( A, x, b, M, restrt, max_it, tol )

# %  -- Iterative template routine --
# %     Univ. of Tennessee and Oak Ridge National Laboratory
# %     October 1, 1993
# %     Details of this algorithm are described in "Templates for the
# %     Solution of Linear Systems: Building Blocks for Iterative
# %     Methods", Barrett, Berry, Chan, Demmel, Donato, Dongarra,
# %     Eijkhout, Pozo, Romine, and van der Vorst, SIAM Publications,
# %     1993. (ftp netlib2.cs.utk.edu; cd linalg; get templates.ps).
# %
# % [x, error, iter, flag] = gmres( A, x, b, M, restrt, max_it, tol )
# %
# % gmres.m solves the linear system Ax=b
# % using the Generalized Minimal residual ( GMRESm ) method with restarts .
# %
# % input   A        REAL nonsymmetric positive definite matrix
# %         x        REAL initial guess vector
# %         b        REAL right hand side vector
# %         M        REAL preconditioner matrix
# %         restrt   INTEGER number of iterations between restarts
# %         max_it   INTEGER maximum number of iterations
# %         tol      REAL error tolerance
# %
# % output  x        REAL solution vector
# %         error    REAL error norm
# %         iter     INTEGER number of iterations performed
# %         flag     INTEGER: 0 = solution found to tolerance
# %                           1 = no convergence given max_it

#    iter = 0;                                         % initialization
#    flag = 0;

#    bnrm2 = norm( b );
#    if  ( bnrm2 == 0.0 ), bnrm2 = 1.0; end

#    r = M \ ( b-A*x );
#    error = norm( r ) / bnrm2;
#    if ( error < tol ) return, end

#    [n,n] = size(A);                                  % initialize workspace
#    m = restrt;
#    V(1:n,1:m+1) = zeros(n,m+1);
#    H(1:m+1,1:m) = zeros(m+1,m);
#    cs(1:m) = zeros(m,1);
#    sn(1:m) = zeros(m,1);
#    e1    = zeros(n,1);
#    e1(1) = 1.0;

#    for iter = 1:max_it,                              % begin iteration

#       r = M \ ( b-A*x );
#       V(:,1) = r / norm( r );
#       s = norm( r )*e1;
#       for i = 1:m,                                   % construct orthonormal
# 	 w = M \ (A*V(:,i));                         % basis using Gram-Schmidt
# 	 for k = 1:i,
# 	   H(k,i)= w'*V(:,k);
# 	   w = w - H(k,i)*V(:,k);
# 	 end
# 	 H(i+1,i) = norm( w );
# 	 V(:,i+1) = w / H(i+1,i);
# 	 for k = 1:i-1,                              % apply Givens rotation
#             temp     =  cs(k)*H(k,i) + sn(k)*H(k+1,i);
#             H(k+1,i) = -sn(k)*H(k,i) + cs(k)*H(k+1,i);
#             H(k,i)   = temp;
# 	 end
# 	 [cs(i),sn(i)] = rotmat( H(i,i), H(i+1,i) ); % form i-th rotation matrix
#          temp   = cs(i)*s(i);                        % approximate residual norm
#          s(i+1) = -sn(i)*s(i);
# 	 s(i)   = temp;
#          H(i,i) = cs(i)*H(i,i) + sn(i)*H(i+1,i);
#          H(i+1,i) = 0.0;
# 	 error  = abs(s(i+1)) / bnrm2;
# 	 if ( error <= tol ),                        % update approximation
# 	    y = H(1:i,1:i) \ s(1:i);                 % and exit
#             x = x + V(:,1:i)*y;
# 	    break;
# 	 end
#       end

#       if ( error <= tol ), break, end
#       y = H(1:m,1:m) \ s(1:m);
#       x = x + V(:,1:m)*y;                            % update approximation
#       r = M \ ( b-A*x )                              % compute residual
#       s(i+1) = norm(r);
#       error = s(i+1) / bnrm2;                        % check convergence
#       if ( error <= tol ), break, end;
#    end

#    if ( error > tol ) flag = 1; end;                 % converged

# % END of gmres.m

# function [ c, s ] = rotmat( a, b )

# %
# % Compute the Givens rotation matrix parameters for a and b.
# %
#    if ( b == 0.0 ),
#       c = 1.0;
#       s = 0.0;
#    elseif ( abs(b) > abs(a) ),
#       temp = a / b;
#       s = 1.0 / sqrt( 1.0 + temp^2 );
#       c = temp * s;
#    else
#       temp = b / a;
#       c = 1.0 / sqrt( 1.0 + temp^2 );
#       s = temp * c;
#    end

In [ ]:
irreps = (1,1,4,1,3,1,4,4,1,1,3,4,2,1,3,4,1,4,1,3,2,1,1,4)

sym_target = 1

# The 

In [ ]:
@njit
def sym_mult_table(irrepA, irrepB):
    mult_table = np.array([[1,2,3,4],
                           [2,1,4,3],
                           [3,4,1,2],
                           [4,3,2,1]])
 
    return mult_table[irrepA-1, irrepB-1]

@njit
def count_t3A_t3D(irreps):
    ctA = 0
    ctot = 0
    for i in range(len(irreps)):
        for j in range(i+1,len(irreps)):
            for k in range(j+1,len(irreps)):
                for a in range(len(irreps)):
                    for b in range(a+1,len(irreps)):
                        for c in range(b+1,len(irreps)):
                            sym_hole = sym_mult_table(sym_mult_table(irreps[i],irreps[j]),irreps[k])
                            sym_particle = sym_mult_table(sym_mult_table(irreps[a],irreps[b]),irreps[c])
                            sym_exc = sym_mult_table(sym_hole,sym_particle)
                            ctot += 1
                            if sym_exc == sym_target:
                                ctA += 2
    return ctA, ctot

@njit
def count_t3B_t3C(irreps):
    ctB = 0
    ctot = 0
    for i in range(len(irreps)):
        for j in range(i+1,len(irreps)):
            for k in range(len(irreps)):
                for a in range(len(irreps)):
                    for b in range(a+1,len(irreps)):
                        for c in range(len(irreps)):
                            sym_hole = sym_mult_table(sym_mult_table(irreps[i],irreps[j]),irreps[k])
                            sym_particle = sym_mult_table(sym_mult_table(irreps[a],irreps[b]),irreps[c])
                            sym_exc = sym_mult_table(sym_hole,sym_particle)
                            ctot += 1
                            if sym_exc == sym_target:
                                ctB += 2
    return ctB, ctot

@njit
def count_t4A_t4E(irreps):
    ctA = 0
    for i in range(len(irreps)):
        for j in range(i+1,len(irreps)):
            for k in range(j+1,len(irreps)):
                for l in range(k+1,len(irreps)):
                    for a in range(len(irreps)):
                        for b in range(a+1,len(irreps)):
                            for c in range(b+1,len(irreps)):
                                for d in range(c+1,len(irreps)):
                                    sym_hole = sym_mult_table(sym_mult_table(sym_mult_table(irreps[i],irreps[j]),irreps[k]),irreps[l])
                                    sym_particle = sym_mult_table(sym_mult_table(sym_mult_table(irreps[a],irreps[b]),irreps[c]),irreps[d])
                                    sym_exc = sym_mult_table(sym_hole,sym_particle)
                                    if sym_exc == sym_target:
                                        ctA += 2
    return ctA

@njit
def count_t4B_t4D(irreps):
    ctB = 0
    for i in range(len(irreps)):
        for j in range(i+1,len(irreps)):
            for k in range(j+1,len(irreps)):
                for l in range(len(irreps)):
                    for a in range(len(irreps)):
                        for b in range(a+1,len(irreps)):
                            for c in range(b+1,len(irreps)):
                                for d in range(len(irreps)):
                                    sym_hole = sym_mult_table(sym_mult_table(sym_mult_table(irreps[i],irreps[j]),irreps[k]),irreps[l])
                                    sym_particle = sym_mult_table(sym_mult_table(sym_mult_table(irreps[a],irreps[b]),irreps[c]),irreps[d])
                                    sym_exc = sym_mult_table(sym_hole,sym_particle)
                                    if sym_exc == sym_target:
                                        ctB += 2
    return ctB

@njit
def count_t4C(irreps):
    ctC = 0
    for i in range(len(irreps)):
        for j in range(i+1,len(irreps)):
            for k in range(len(irreps)):
                for l in range(k+1,len(irreps)):
                    for a in range(len(irreps)):
                        for b in range(a+1,len(irreps)):
                            for c in range(len(irreps)):
                                for d in range(c+1,len(irreps)):
                                    sym_hole = sym_mult_table(sym_mult_table(sym_mult_table(irreps[i],irreps[j]),irreps[k]),irreps[l])
                                    sym_particle = sym_mult_table(sym_mult_table(sym_mult_table(irreps[a],irreps[b]),irreps[c]),irreps[d])
                                    sym_exc = sym_mult_table(sym_hole,sym_particle)
                                    if sym_exc == sym_target:
                                        ctC += 2
    return ctC

In [ ]:
ctA, ctot1 = count_t3A_t3D(irreps)
ctB, ctot2 = count_t3B_t3C(irreps)
tot = ctot1 + ctot2
num_triples = ctA + ctB

In [ ]:
# num_quadruples = count_t4A_t4E(irreps) + count_t4B_t4D(irreps) + count_t4C(irreps)

In [ ]:
num_quadruples

In [ ]:
num_triples

In [ ]:
1/36*(10*9*8*38*37*36)

In [ ]:
1/(24**2)*(10*9*8*7*38*37*36*35)

In [ ]:
A = np.random.rand(3,3)

In [ ]:
A

In [ ]:
B = np.random.rand(3,2)

In [ ]:
B

In [ ]:
C = np.concatenate((A,B),axis=1)

In [ ]:
C

In [ ]:
D = C[:,:C.shape[1]-B.shape[1]]

In [ ]:
D

In [ ]:
np.einsum('ijab,abij->',lambda2,t2)

In [ ]:
scf_resid